In [1]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import numpy.matlib    
import pandas as pd
import scipy as sc
import csv
import math
import random
from sklearn.model_selection import train_test_split
from scipy.sparse import isspmatrix, csc_matrix, csr_matrix
from scipy.sparse.linalg import eigsh, svds
from scipy.linalg import svd
import os
from sklearn.cluster import KMeans
from sklearn.preprocessing import OneHotEncoder
import io

In [2]:
from skopt import gp_minimize
from skopt.space import Real, Integer

In [3]:
eps                  = np.finfo(float).eps

In [4]:
def createList(r1,r2):
    return [item for item in range(r1,r2 + 1)]

In [5]:
def userSim(Y): 
    print("In userSim")
    N,M              = Y.shape  
    
    Y01              = Y.copy()
    Y01[Y01>0]       = 1
    
    tmp              = (Y**2)@(Y01.T)
    tmp              = tmp + tmp.T
    tmp              = tmp - 2*(Y@(Y.T))
    
    simScore         = 1.0/(1 + np.sqrt(tmp))
    
    nominator        = Y01@Y01.T
    
    denominator      = np.sum(Y01,axis=1).reshape(-1,1)
    denominator      = np.matlib.repmat((denominator),1,N)
    denominator      = denominator + denominator.T
    denominator      = denominator - nominator
    
    B1               = nominator/denominator
    B1[np.isnan(B1)] = 0
    
    simScore         = simScore*B1
    return simScore

In [6]:
def evecs(A,nEvecs):
    npix             = A.shape[0]
    useSparse        = isspmatrix(A)
    
    dd               = 1/((np.sum(A,axis=0))+eps)
    dd               = np.sqrt(dd)

    if(useSparse):
        DD           = sc.sparse.diags(dd)
    else:
        DD           = np.diag(dd)
    
    L                = DD@A@DD
    
    if(useSparse==1):
        ss,v         = eigsh(L,nEvecs,sigma=1)
        ss           = np.flip(ss)
        V            = np.flip(v,axis=1)
    else:
        u,ss,v       = svd(L)
        V            = u[:, :nEvecs]
        
    ss               = ss.reshape(-1,1)
    ss               = ss[:nEvecs]
    
    return V,ss

In [7]:
def spectralClu(A=None,K=None): 
    # A: Affinity Matrix, Higher value -> more similar
    # K: Number of CLuster
    n                = A.shape[0]
    
    D                = np.sum(A,axis=1).reshape(-1,1)
    D[D==0]          = eps
    D                = sc.sparse.spdiags((1.0/np.sqrt(D)).T,0,n,n)
    
    L                = D@A@D
    V,evals          = evecs(L,K)
    
    Vnorm            = (np.linalg.norm(V,axis=1) + eps).reshape(-1,1)
    
    V                = V/Vnorm

    kmeans           = KMeans(n_clusters=K,max_iter=maxiter,random_state=0).fit(V)
    idx              = kmeans.predict(V)

    return idx

In [8]:
def aggregratePrediction(X,UGidx):
    n,m                  = X.shape
    K1                   = len(np.unique(UGidx))
    ctmp_d               = np.ones(n)
    ctmp                 = csc_matrix((ctmp_d,(UGidx.reshape(-1,),range(n))),shape=(K1,n))

    GFreq                = np.bincount(UGidx.reshape(-1,)).reshape(-1,1)
    XPerA                = (ctmp@X)/np.matlib.repmat((GFreq),1,m)
    
    
    return XPerA

In [9]:
def MAE(a,b):
    return np.sum(abs(a*(b!=0) - b))/np.sum(b!=0)

In [10]:
def RMSE(X,Y):
    tmp                        = ((Y - X)*(Y!=0))**2
    
    return math.sqrt(np.sum(tmp)/np.sum(Y!=0))

In [11]:
def AUC(Ypre,Yt,cutoff):
    userWithRating             = (np.sum(Yt!=0,axis=1)>0)
    Yt                         = Yt[userWithRating,:]
    Ypre                       = Ypre[userWithRating,:]
    N,M                        = Yt.shape
    auc                        = 0
    uWithPair                  = 0
    for user in range(N):
        userRating             = Yt[user,:]
        userPrediction         = Ypre[user, :]
        ratedIdx               = userRating>0
        binUserRating          = 2*(userRating[ratedIdx]>=cutoff)-1
        predForKnown           = userPrediction[ratedIdx]
        
        posIdx                 = np.where(binUserRating==1)
        posIdx                 = posIdx[0].T

        negIdx                 = np.where(binUserRating==-1)
        negIdx                 = negIdx[0]
        
        if(np.size(posIdx)!=0 and np.size(negIdx)!=0):
            negIdxGrid         = np.matlib.repmat(negIdx,len(posIdx),1)
            ttlNeg             = np.size(negIdxGrid)
            negIdxGrid         = negIdxGrid.reshape(ttlNeg,1,order='F')
            posIdxGrid         = np.matlib.repmat(posIdx.reshape(-1,1),len(negIdx),1)
            pairs              = np.concatenate((negIdxGrid,posIdxGrid),axis=1)
            pairsPred          = np.concatenate((pairs,predForKnown[negIdxGrid],predForKnown[posIdxGrid]),axis = 1)
            pairsPredWithScore = np.concatenate([pairsPred,(0.5*(pairsPred[:,2]==pairsPred[:,3]).reshape(-1,1)),(1*(pairsPred[:,2]<pairsPred[:,3]).reshape(-1,1))],axis=1)
            auc                = auc + (np.sum(pairsPredWithScore[:,4:6]))/pairsPredWithScore.shape[0]
            uWithPair          = uWithPair + 1    
    auc                        = auc/uWithPair
    
    return auc

In [12]:
def precAtK(Ypre,Yt,k,cutoff):
    # %Yt  : Test Set of size n *m
    # %Ypre: Prediction set of size n *m
    # %cutoff : threshold for relevant item
    # %k: precision@k

    userWithRating             = (np.sum(Yt!=0,axis=1)>0)
    Yt                         = Yt[userWithRating,:]
    Ypre                       = Ypre[userWithRating,:]
    N,M                        = Yt.shape
    prec                       = np.zeros((1,k))

    for user in range(N):
        userRating             = Yt[user, :]
        
        ratedRelevantIdx       = np.where(userRating>=cutoff)
        ratedRelevantIdx       = ratedRelevantIdx[0]

        ratedIdx               = np.where(userRating!=0)
        ratedIdx               = ratedIdx[0]

        userPrediction         = Ypre[user, :]
        prediction             = userPrediction[ratedIdx]
        sortesPIdxTmp          = np.argsort(prediction)[::-1]
        sortesPIdx             = ratedIdx[sortesPIdxTmp]
        nz                     = len(sortesPIdx)
        for kNo in range(k):
            intrsct            = np.intersect1d(sortesPIdx[0:(min(kNo,nz)+1)],ratedRelevantIdx)
            mx                 = max(min(kNo+1,nz),eps)
            prec[:,kNo]        = prec[:,kNo] + len(intrsct)/mx
            
    prec                       = prec/N
    
    return prec

In [13]:
def recallAtK(Ypre,Yt, k, cutoff):
    # %Yt  : Test Set of size n *m
    # %Ypre: Prediction set of size n *m
    # %cutoff : threshold for relevant item
    # %k: recall@k

    userWithRating             = (np.sum(Yt!=0,axis=1)>0)
    Yt                         = Yt[userWithRating,:]
    Ypre                       = Ypre[userWithRating,:]
    N,M                        = Yt.shape
    recall                     = np.zeros((1,k))
    
    for user in range(N):
        userRating             = Yt[user, :]
        userPrediction         = Ypre[user, :]
        
        ratedRelevantIdx       = np.where(userRating>=cutoff)
        ratedRelevantIdx       = ratedRelevantIdx[0]

        ratedIdx               = np.where(userRating!=0)
        ratedIdx               = ratedIdx[0]

        prediction             = userPrediction[ratedIdx]
        
        sortesPIdxTmp          = np.argsort(prediction)[::-1]
        sortesPIdx             = ratedIdx[sortesPIdxTmp]
        
        nz                     = len(sortesPIdx)
        for kNo in range(k):
            intrsct            = np.intersect1d(sortesPIdx[0:(min(kNo,nz)+1)],ratedRelevantIdx)
            mx                 = max(len(ratedRelevantIdx),eps)
            recall[:,kNo]      = recall[:,kNo] + len(intrsct)/mx
            
    recall                     = recall/N
    
    return recall 

In [14]:
def ndcgAtk(Ypre,Yt,k):
    # %Yt  : Test Set of size n *m
    # %Ypre: Prediction set of size n *m
    # %cutoff : threshold for relevant item
    # %k: precision@k

    res                        = np.zeros((1,k))
    cnt                        = 0
    N,M                        = Yt.shape

    for user in range(N):
        ratedIdx               = Yt[user,:]!=0

        userRating             = Yt[user,ratedIdx]
        userPrediction         = Ypre[user,ratedIdx]
        nz                     = len(userRating)

        ranks                  = np.zeros((1,nz))
        ideal_ranks            = np.zeros((1,nz))
        
        I                      = np.argsort(-userPrediction,axis=0)
        ideal_I                = np.argsort(-userRating,axis=0)

        ranks                  = I
        ideal_ranks            = ideal_I

        oriOrder               = np.argsort(ideal_ranks,axis=0)

        nominator              = userRating/(np.log(ranks+2))
        denominator            = userRating/(np.log(ideal_ranks + 2))
    
        nominator              = nominator[oriOrder]
        denominator            = denominator[oriOrder]
        
        if k > nz:
            nominator          = np.concatenate((nominator, np.zeros((k - nz))))
            denominator        = np.concatenate((denominator, np.zeros((k - nz))))
        elif k < nz:
            nominator          = nominator[0:k]
            denominator        = denominator[0:k]          

        if np.array(np.where(np.cumsum(denominator)== 0)).shape[1] != 0:
            tmp                = np.zeros((1,k))
        else:
            tmp                = np.cumsum(nominator) / np.cumsum(denominator)
            cnt                = cnt + 1
        
        res                    = res + tmp
        
    res                        = res/cnt
    
    return res

In [15]:
def EvaluationAllUpdated(Yprd,Y1,k,cutoff):
    mae                        = MAE(Yprd, Y1) 
    rmse                       = RMSE(Yprd, Y1)
    auc                        = AUC(Yprd, Y1, cutoff)
    precision                  = precAtK(Yprd,Y1, k, cutoff)
    recall                     = recallAtK(Yprd,Y1, k, cutoff)
    f1                         = (2*precision*recall)/(precision + recall + eps)
    ndcg                       = ndcgAtk(Yprd,Y1,k)

    return mae,rmse,auc,precision,recall,f1,ndcg

In [16]:
def EvaluationAllUpdated_N(Yprd,Y1,k,cutoff):
    mae                        = MAE(Yprd, Y1) 
    rmse                       = RMSE(Yprd, Y1)

    return mae,rmse

In [17]:
cols                     = ["Model","MAE","RMSE","AUC","Precision@1","Precision@5","Precision@10","Precision@20",
                            "Precision@30","Precision@40","Recall@1","Recall@5","Recall@10","Recall@20","Recall@30",
                            "Recall@40","F1@1","F1@5","F1@10","F1@20","F1@30","F1@40","NDCG@1","NDCG@5","NDCG@10",
                            "NDCG@20","NDCG@30","NDCG@40"]
Result_Trn               = pd.DataFrame(columns=cols)

In [18]:
cols                     = ["Model","MAE","RMSE"]
Result_Trn_CDR               = pd.DataFrame(columns=cols)
Result_Tst_CDR = pd.DataFrame(columns=cols)

In [19]:
def gradUnifiedLS(v,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx): 
    
    n,m                  = Y.shape
    U                    = v[0:n*d].reshape(n,d,order='F')
    V                    = v[n*d:n*d+m*d].reshape(m,d,order='F')
    UG                   = v[n*d+m*d:n*d+m*d+K1*d].reshape(K1,d,order='F')
    VG                   = v[n*d+m*d+K1*d:].reshape(K2,d,order='F')

    UGmatMU              = (UG[UGidx.reshape(-1,),:]) - U
    VGmatMV              = (VG[VGidx.reshape(-1,),:]) - V
    
    X                    = U@(V.T)
    Ygt0                 = Y>0
    YMXgt0               = (Y - X)*Ygt0

    regobj               = (lambda3/2)*(np.sum(U**2) + np.sum(V**2))

    lossobj              = 0
    lossobj              = lossobj + (0.5*(np.sum(YMXgt0**2)))
    lossobj              = lossobj + (lambda1/2)*(np.sum(UGmatMU**2) + np.sum(VGmatMV**2))
    
    dU                   = lambda3*U
    dV                   = lambda3*V
    
    dU                   = dU - YMXgt0@V
    dV                   = dV - (YMXgt0.T)@U

    dU                   = dU - (lambda1*UGmatMU)
    dV                   = dV - (lambda1*VGmatMV)

    ctmp_d               = np.ones(n)                   # convert UGidx to a K1-by-n matrix containing the k1 indicator vectors as row
    ctmp                 = csc_matrix((ctmp_d,(UGidx.reshape(-1,),range(n))),shape=(K1,n))
    dUG                  = lambda1*((ctmp)@UGmatMU)
    
    ctmp_d               = np.ones(m)                   # convert UGidx to a K2-by-m matrix containing the k2 indicator vectors as row
    ctmp                 = csc_matrix((ctmp_d,(VGidx.reshape(-1,),range(m))),shape=(K2,m))
    dVG                  = lambda1*((ctmp)@VGmatMV)
    
    obj                  = regobj + lossobj
   
    grad                 = np.concatenate((dU.flatten('F'),dV.flatten('F'),dUG.flatten('F'),dVG.flatten('F'))).reshape(-1,1)
    
    return obj,grad,lossobj,regobj

In [20]:
def gradUnifiedMMMF(v,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGIdx,VGIdx):

    n,m                  = Y.shape
    U                    = v[0:n*d].reshape(n,d,order='F')
    V                    = v[n*d:n*d+m*d].reshape(m,d,order='F')
    theta                = v[n*d+m*d:n*d+m*d+n*(l-1)].reshape(n,l-1,order='F')
    UG                   = v[n*d+m*d+n*(l-1):n*d+m*d+n*(l-1)+K1*d].reshape((K1,d), order='F')
    VG                   = v[n*d+m*d+n*(l-1)+K1*d:n*d+m*d+n*(l-1)+K1*d+K2*d].reshape((K2,d), order='F')
    thetaG               = v[n*d+m*d+n*(l-1)+K1*d+K2*d:].reshape((K1,l-1), order='F')
        
    UGmatMU              = (UG[UGIdx.reshape(-1,),:]) - U
    VGmatMV              = (VG[VGIdx.reshape(-1,),:]) - V
    thetaGMtheta         = (thetaG[UGIdx.reshape(-1,),:]) - theta
    
    X                    = U@V.T
    Ygt0                 = Y>0
    BX                   = X*Ygt0
    
    dU                   = lambda3*U
    dV                   = lambda3*V
    dtheta               = np.zeros((n, l-1))
    
    regobj               = (lambda3/2)*(np.sum(U**2) + np.sum(V**2))
    
    lossobj              = 0
    lossobj              = lossobj + (lambda1/2)*(np.sum(UGmatMU**2) + np.sum(VGmatMV**2) + np.sum(thetaGMtheta**2))
    
    for k in range(l-1):
        S                = Ygt0 - 2*(Y>k+1)
        BZ               = (theta[:,k].reshape(-1,1)@(np.ones([1,m])))*S - BX*S
        lossobj          = lossobj + sum(sum(h(BZ)))
        tmp              = hprime(BZ)*S
        dU               = dU - tmp@V
        dV               = dV - tmp.T@U
        dtheta[:,k]      = tmp@np.ones((m,))
        
    dU                   = dU - lambda1*UGmatMU
    dV                   = dV - lambda1*VGmatMV
    dtheta               = dtheta - lambda1*thetaGMtheta
    
    ctmp_d               = np.ones(n)                   # convert UGidx to a K1-by-n matrix containing the k1 indicator vectors as row
    ctmp                 = csc_matrix((ctmp_d,(UGIdx.reshape(-1,),range(n))),shape=(K1,n))
    dUG                  = np.multiply(lambda1,(ctmp@UGmatMU))
    dthetaG              = np.multiply(lambda1,(ctmp@thetaGMtheta))
 
    ctmp_d               = np.ones(m)                   # convert UGidx to a K2-by-m matrix containing the k2 indicator vectors as row
    ctmp                 = csc_matrix((ctmp_d,(VGIdx.reshape(-1,),range(m))),shape=(K2,m))
    dVG                  = np.multiply(lambda1,(ctmp@VGmatMV))
    
    obj                  = regobj + lossobj;            # obj is the objective function that we need to minimize

    grad                 = np.concatenate((dU.flatten('F'),dV.flatten('F'),dtheta.flatten('F'),dUG.flatten('F'),dVG.flatten('F'),dthetaG.flatten('F'))).reshape(-1,1)
    
    return obj,grad,lossobj,regobj

In [21]:
def conjgrad(x0,objGrad,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx):
    J                     = []
    temp                  = 0
    nu                    = 0.1
    abstol                = 0                                 # stop if gradient magnitude goes below this
    allowNonDecrease      = 0                                 # don't stop if line search fails to find decrease
    digits                = 12                                # digits of precision to use for objective comparisons
    ogfun                 = objGrad
    ogcalls               = 0
    x                     = x0
    numiter               = 0
    j                     = 0
    alpha                 = 10**(-10)
    ogcalls               = ogcalls + 1
    obj,dx,lossobj,regobj = ogfun(x,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx)
    r                     = -dx
    s                     = r
    dirn                  = s
    deltanew              = (r.T)@dirn
    deltazero             = deltanew
    
    while ((numiter<maxiter) and (abs(deltanew)>tol*tol*abs(deltazero)) and (abs(deltanew)>abstol)):
        numiter           = numiter + 1
        j                 = j + 1
        print('\n %.4f' %obj)
        J.append(obj)
        prevobj           = obj

        if (alpha < 10**(-10)):
            alpha         = 10**(-10)

        alpha,obj,dx,ogc  = cgLineSearch(x,obj,dx,dirn,alpha,objGrad,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx,lossobj,regobj)
        ogcalls           = ogcalls + ogc
        
        temp              = temp + alpha
        x                 = x + alpha*dirn
        r                 = -dx
        deltaold          = deltanew
        deltamid          = r.T@s
        deltanew          = r.T@r
        beta              = (deltanew - deltamid) / deltaold
        dirn              = r + max(0, beta)*dirn
        if ((deltamid/deltanew >= nu) or (dirn.T@dx >= 0)):
            dirn          = r
            j             = 0
        s                 = r
        print(numiter)
        
    return x,numiter,ogcalls,J

In [22]:
def cgLineSearch(x0,obj0,dx0,direction,alpha,objGrad,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx,lobj,robj):
    seciter                   = 5                               # maximum number of quadratic interpolation iterations
    alpha0                    = alpha
    c1                        = 10**(-4)                        # required decrease in objective (relative to gradient)
    digits                    = 12                              # digits of precision to use for objective comparisons
    gamma                     = 10
    ogfun                     = objGrad
    
    if (alpha0 <= 0):                                           # check initial values
        print('alpha0 must be greater than zero')
    
    obj                       = obj0                            # begin line search
    dx                        = dx0
    etazero                   = (dx.T)@direction
    etaprev                   = etazero
    alpha                     = alpha0
    ogcalls                   = 0
    lossobj                   = lobj
    regobj                    = robj
    
    alpha,obj,dx,lossobj,regobj,ogcalls = findNonZeroAlpha(x0,alpha,direction,ogfun,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx,ogcalls,obj,dx,lossobj,regobj)
    
    obj,dx,lossobj,regobj     = ogfun(x0+alpha*direction,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx)
    ogcalls                   = ogcalls + 1
    
    oldalpha,oldobj,olddx                           = saveAlpha(alpha,obj,dx)
    alpha,obj,dx,lossobj,regobj,ogcalls,doBacktrack = backtrack(x0,alpha,direction,ogfun,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx,ogcalls,gamma,obj,dx,lossobj,regobj,digits,obj0,c1,etazero,oldalpha,oldobj,olddx)

    beta                      = alpha
    eta                       = dx.T@direction
    i                         = 0
    
    while((abs(eta)>c2*abs(etazero)) and (i<seciter) and (pround(obj,digits) <= pround(obj0,digits)) and (etaprev!=eta) and (np.sum((x0 + alpha*direction) != x0)>0)):
        beta                  = eta*beta / (etaprev - eta)
        oldalpha,oldobj,olddx = saveAlpha(alpha,obj,dx)
        alpha                 = alpha + beta
        if(alpha<=0):
            alpha             = 1
        etaprev               = eta
        i                     = i + 1
        obj,dx,lossobj,regobj = ogfun(x0+alpha*direction,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx)
        ogcalls               = ogcalls + 1
        eta                   = dx.T@direction

    alpha,obj,dx,lossobj,regobj,ogcalls             = findNonZeroAlpha(x0,alpha,direction,ogfun,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx,ogcalls,obj,dx,lossobj,regobj)
    alpha,obj,dx,lossobj,regobj,ogcalls,doBacktrack = backtrack(x0,alpha,direction,ogfun,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx,ogcalls,gamma,obj,dx,lossobj,regobj,digits,obj0,c1,etazero,oldalpha,oldobj,olddx)
    checkConditions(obj,digits,obj0,etazero,eta,x0,alpha,direction)
    
    return alpha,obj,dx,ogcalls 

In [23]:
def findNonZeroAlpha(x0,alpha,direction,objGrad,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx,ogcalls,obj,dx,lossobj,regobj):
    if(np.array_equal((x0+alpha*direction),x0)):                # Make sure alpha isn't smaller than level of precision
        preAlpha              = alpha
        
        while(np.array_equal((x0+alpha*direction),x0)):
            alpha             = alpha*gamma
        
        obj,dx,lossobj,regobj = objGrad(x0+alpha*direction,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx)
        ogcalls               = ogcalls + 1
        
    return alpha,obj,dx,lossobj,regobj,ogcalls

In [24]:
def backtrack(x0,alpha,direction,objGrad,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx,ogcalls,gamma,obj,dx,lossobj,regobj,digits,obj0,c1,etazero,oldalpha,oldobj,olddx):
    #ensures either (1) Armijo lowex0,alpha,direction,objGrad,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx,ogcallsr objective, or (2) infinitesimal alpha
    doBacktrack               = 0
    preAlpha                  = alpha
    while((pround(obj,digits)>pround(obj0 + c1*alpha*etazero, digits)) and (np.sum((x0+alpha*direction)!=x0)>0)):
        if(ogcalls > 1):
            if((oldalpha > alpha/gamma) and (oldobj < (obj0+c1*oldalpha*etazero))):
                alpha,obj,dx  = restoreAlpha(oldalpha,oldobj,olddx)
                doBacktrack   = 1
                break
        alpha                 = alpha/gamma
        obj,dx,lossobj,regobj = objGrad(x0+alpha*direction,c2,tol,maxiter,l,d,Y,lambda1,lambda3,K1,K2,UGidx,VGidx)
        ogcalls               = ogcalls + 1
        doBacktrack           = 1
    return alpha,obj,dx,lossobj,regobj,ogcalls,doBacktrack

In [25]:
def saveAlpha(alpha,obj,dx):
    #make sure current step yields decrease in objective
    oldalpha   = alpha
    oldobj     = obj
    olddx      = dx
    return oldalpha,oldobj,olddx

In [26]:
def restoreAlpha(oldalpha,oldobj,olddx):
    alpha      = oldalpha
    obj        = oldobj
    dx         = olddx
    return alpha,obj,dx

In [27]:
def pround(x,d):
    d          = round(d)
    if (d<1):
        print('Number of digits must be integer d = %.4e \n'%d)
    if(x==0 or math.isnan(x) or math.isinf(x)):
        y      = x   
    else:
        p      = math.floor(math.log10(abs(x)))+1
        factor = 10**(d-p)
        y      = np.round(x*factor)/factor
    return y

In [28]:
def checkConditions(obj,digits,obj0,etazero,eta,x0,alpha,direction):
    if round(obj, digits) >= round(obj0, digits):
        print('Warning: Finished line search without decreasing objective.\nMay have reached limit of precision, or obj/grad code may be broken.\n')
    if etazero == eta:
        print('Warning: Line search yielded no change in directional derivative.\nMay have reached limit of precision.\n')
    if sum(x0 + alpha * direction != x0) == 0:
        print('Warning: Line search yielded no change in position.\nMay have reached limit of precision.\n')

In [29]:
def h(z):
    zin01      = (z>0)^(z>=1)
    zle0       = z<0
    ret        = zin01/2 - zin01*z + zin01*(z**2)/2 + zle0/2 - zle0*z
    return ret  

In [30]:
def hprime(z):
    zin01      = (z>0)^(z>=1)
    zle0       = z<0
    ret        = zin01*z - zin01 - zle0
    return ret

In [31]:
def m3fSoftmax(xy,theta):
    n,m        = xy.shape
    n1,l1      = theta.shape
    if(n!=n1):
        print('sizes of xy and theta don''t match');
    y          = np.ones((n,m))
    for i in range(l1):
        tmp    = ((theta[:,i]).reshape(-1,1))@(np.ones((1,m)))
        tmp    = xy>=tmp
        y      = y + tmp
    return y

## Loading Matrices for Video Games domain

In [32]:
from pandas import read_csv

In [33]:
rating_matrix_video_6k = np.loadtxt("rating_matrix_video_6k.csv", delimiter=",")

In [34]:
temp_d = read_csv('new_user_ids_video_6k.csv')
temp_np = temp_d.to_numpy()
temp_np = np.delete(temp_np, 0, 1)
new_user_ids_video_6k = temp_np.reshape(temp_np.shape[0],)

In [35]:
temp_d = read_csv('item_ids_video_6k_rated_unrated.csv')
temp_np = temp_d.to_numpy()
temp_np = np.delete(temp_np, 0, 1)
item_ids_video_6k_rated_unrated = temp_np.reshape(temp_np.shape[0],)

## Loading Matrics for Movies and TV domain

In [211]:
rating_matrix_mt_6k = np.loadtxt("rating_matrix_mt_6k.csv", delimiter=",")

In [212]:
temp_d = read_csv('new_user_ids_mt_6k.csv')
temp_np = temp_d.to_numpy()
temp_np = np.delete(temp_np, 0, 1)
new_user_ids_mt_6k = temp_np.reshape(temp_np.shape[0],)

In [213]:
temp_d = read_csv('item_ids_mt_6k_rated_unrated.csv')
temp_np = temp_d.to_numpy()
temp_np = np.delete(temp_np, 0, 1)
item_ids_mt_6k_rated_unrated = temp_np.reshape(temp_np.shape[0],)

## Preprocessing Video Games domain

In [33]:
names = ['user_id', 'game_id', 'rating', 'timestamp']
df    = pd.read_csv('ratings_Video_Games.csv', sep=',', names=names)
df.head()

,user_id,game_id,rating,timestamp
0,AB9S9279OZ3QO,0078764343,5.0,1373155200
1,A24SSUT5CSW8BH,0078764343,5.0,1377302400
2,AK3V0HEBJMQ7J,0078764343,4.0,1372896000
3,A10BECPH7W8HM7,043933702X,5.0,1404950400
4,A2PRV9OULX1TWP,043933702X,5.0,1386115200


In [34]:
n_users_video = df.user_id.nunique(dropna = True)
m_items_video = df.game_id.nunique(dropna = True)
print(str(n_users_video) + " " + str(m_items_video))

826767 50210


In [35]:
user_ids_video = df.user_id.unique()
item_ids_video = df.game_id.unique()
print(user_ids_video.shape)
print(item_ids_video.shape)

(826767,)
(50210,)


## Preprocessing Movies & TV domain

In [36]:
names = ['user_id', 'movie_id', 'rating', 'timestamp']
df    = pd.read_csv('ratings_Movies_and_TV.csv', sep=',', names=names)
df.head()

,user_id,movie_id,rating,timestamp
0,A3R5OBKS7OM2IR,0000143502,5.0,1358380800
1,A3R5OBKS7OM2IR,0000143529,5.0,1380672000
2,AH3QC2PC1VTGP,0000143561,2.0,1216252800
3,A3LKP6WPMP9UKX,0000143588,5.0,1236902400
4,AVIY68KEPQ5ZD,0000143588,5.0,1232236800


In [37]:
n_users_mt = df.user_id.nunique(dropna = True)
m_items_mt = df.movie_id.nunique(dropna = True)
print(str(n_users_mt) + " " + str(m_items_mt)) 

2088620 200941


In [38]:
user_ids_mt = df.user_id.unique()
item_ids_mt = df.movie_id.unique()
print(user_ids_mt.shape)
print(item_ids_mt.shape)

(2088620,)
(200941,)


## Dataset trimming

In [39]:
overlap = 6000

In [40]:
print(user_ids_video.shape)
print(user_ids_mt.shape)

(826767,)
(2088620,)


In [41]:
overlapping_user_ids = np.intersect1d(user_ids_video, user_ids_mt)
print(overlapping_user_ids)
print(overlapping_user_ids.shape)

['A0002090WKEMAO8KOWKM' 'A00230923E4Y7VHWZK0IC' 'A002439424KGHR3LZ1OMZ'
 ... 'AZZW55XIQD8QC' 'AZZY72H9Y2F6O' 'AZZZ9DDUPKNKC']
(155797,)


In [42]:
non_overlapping_user_ids_video = np.array(list(set(user_ids_video.tolist()).difference(set(overlapping_user_ids.tolist()))))
non_overlapping_user_ids_mt = np.array(list(set(user_ids_mt.tolist()).difference(set(overlapping_user_ids.tolist()))))
print(non_overlapping_user_ids_video.shape)
print(non_overlapping_user_ids_mt.shape)

(670970,)
(1932823,)


In [43]:
req_overlap_user_ids = np.random.choice(overlapping_user_ids, size=overlap, replace=False)
print(req_overlap_user_ids.shape)
# test = np.array(list(set(overlapping_user_ids_t.tolist()).difference(set(train.tolist()))))

(6000,)


In [44]:
req_non_overlap_user_ids_video = np.random.choice(non_overlapping_user_ids_video, size= 9000 - overlap, replace=False)
req_non_overlap_user_ids_mt = np.random.choice(non_overlapping_user_ids_mt, size= 7000 - overlap, replace=False)
print(req_non_overlap_user_ids_video.shape)
print(req_non_overlap_user_ids_mt.shape)

(3000,)
(1000,)


In [46]:
new_user_ids_video_6k = np.concatenate((req_overlap_user_ids, req_non_overlap_user_ids_video), axis=None)
new_user_ids_mt_6k = np.concatenate((req_overlap_user_ids, req_non_overlap_user_ids_mt), axis=None)
np.random.shuffle(new_user_ids_video_6k)
np.random.shuffle(new_user_ids_mt_6k)
print(new_user_ids_video_6k.shape)
print(new_user_ids_mt_6k.shape)

(9000,)
(7000,)


In [47]:
print(new_user_ids_video_6k)
print(new_user_ids_mt_6k)

['A3ND8NK4MIEZ7D' 'A1837MW9OEYZDY' 'A2R332FUOEGOWS' ... 'A2P0L2XE35BYQW'
 'A1LOI975LTN5XB' 'A1ZHOPK6UMXXBC']
['A2OGE60OKOFMQF' 'A1N5ZBZRQMCNZD' 'A1VGM7F6OQEFPC' ... 'A14SWRWHBBJ7AQ'
 'A2QS7PBWB8JU6Q' 'A3T99VA49A8FHQ']


In [48]:
overlapping_user_ids_t = np.intersect1d(new_user_ids_video_6k, new_user_ids_mt_6k)
print(overlapping_user_ids_t)
print(overlapping_user_ids_t.shape)

['A01632683TJ1GCADQ8B7X' 'A031533437UL5KXSH8FNB' 'A03550173UTIP5TNQ9T7P'
 ... 'AZWM4WE39FNY3' 'AZXUFTJVXJWL0' 'AZZOUDTWW6ZYZ']
(6000,)


## Create rating matrix for video games domain

In [49]:
names = ['user_id', 'game_id', 'rating', 'timestamp']
df    = pd.read_csv('ratings_Video_Games.csv', sep=',', names=names)
df.head()

,user_id,game_id,rating,timestamp
0,AB9S9279OZ3QO,0078764343,5.0,1373155200
1,A24SSUT5CSW8BH,0078764343,5.0,1377302400
2,AK3V0HEBJMQ7J,0078764343,4.0,1372896000
3,A10BECPH7W8HM7,043933702X,5.0,1404950400
4,A2PRV9OULX1TWP,043933702X,5.0,1386115200


In [50]:
df = df.loc[df['user_id'].isin(new_user_ids_video_6k.tolist())] 

In [51]:
df.shape

(19052, 4)

In [52]:
present_rating = df.rating.unique()
print(present_rating)

[4. 5. 3. 1. 2.]


In [53]:
new_item_ids_video_6k = df.game_id.unique()
print(new_item_ids_video_6k.shape)
print(new_item_ids_video_6k)

(9245,)
['0439591368' '0700026657' '0700099867' ... 'B00KGD69U6' 'B00KGGJPX6'
 'B00L4KVCYS']


In [54]:
print(item_ids_video.shape)

(50210,)


In [55]:
item_ids_video_6k_rated_unrated = new_item_ids_video_6k
np.random.shuffle(item_ids_video_6k_rated_unrated)
print(item_ids_video_6k_rated_unrated.shape)

(9245,)


In [56]:
rating_matrix_video_6k = np.zeros([9000, item_ids_video_6k_rated_unrated.shape[0]], dtype='float32')
print(rating_matrix_video_6k.shape)

(9000, 9245)


In [57]:
for ind in df.index:
    if df['user_id'][ind] in new_user_ids_video_6k:
        rating_matrix_video_6k[np.where(new_user_ids_video_6k == df['user_id'][ind])[0][0]][np.where(item_ids_video_6k_rated_unrated == df['game_id'][ind])[0][0]] = df['rating'][ind]
        print(str(df['user_id'][ind]) + " " + str(df['game_id'][ind]) + " " + str(df['rating'][ind]))

A25C0NYV8WD775 0439591368 4.0
A2VLH0R0UP9ULJ 0700026657 5.0
A228QXX35ISH3K 0700099867 5.0
A2JN6SFNKAUAKH 0758534531 3.0
A7NJVEL9E3L0H 1594590133 5.0
A22WTD36WE9YF8 3866811659 4.0
AMUE28FR1I73K 7537823413 5.0
A2BCP0H3RW6VMU 7543450933 5.0
A9H4PYUMOXHXP 7543450933 5.0
A3OUIV77UF406W 8565000168 3.0
A3LOAQX4DVQPCS 8990289440 4.0
A22JJTUEICR3ZR 907843905X 5.0
A1TR4ESRA2C3NP 9625990674 4.0
A34YTZ6R8IIQOM 9625990674 5.0
A1LC9TKGZ046K0 9625992413 5.0
A29LGDX5HC91T7 9861019731 5.0
A1QC81EF99RSPK 9861019731 1.0
A2JD7DAXVR587H 9861127380 5.0
A26A9FFNCGQVDF B000006OVI 4.0
ANIQ8Y26SJRTB B000006OVI 2.0
ATOMYGMSU90LF B000006OVI 4.0
A2SKE0TSEPYPDK B000006OVK 3.0
APQ8ZK7Z3BYVS B000006OVK 5.0
A3KMEUY129LFLH B000006OWR 5.0
A3DII4J55ZGA9B B000006OWT 1.0
A3GKPMHV4U64IY B000006RGO 4.0
ACMO0RPLWYRHV B000006RGQ 5.0
A2R10K5Y1H8L4S B000006RGR 5.0
A1KWOG8QCVSGYX B000006RGR 5.0
A3D6DPWQW509R9 B000007VDN 5.0
AT4LIIE1NCFZI B00000DMA8 4.0
A2MQ5O07UDJDOA B00000DMA8 5.0
A2DHMQDGLYPER0 B00000DMAG 5.0
A33MOM8PD59MKW B00

A145JIAR9QMSYI B0000507UR 5.0
AZ16IZGQ3WLTZ B000050FAG 2.0
A2ZBHG85BL6K3C B000050FBJ 4.0
ABBSVR2KC1H6A B000050FBJ 5.0
A2PJFG5U89NJ35 B000050FBJ 5.0
A1AUUSFH6FD6MA B000050FBJ 3.0
A3B9YPMVHDUXSU B000050FBJ 5.0
AO9WIXEG2A4UK B000050FBJ 5.0
AEPL5QZUY412N B000050I88 5.0
A4ZS08A5VLP13 B000050NRN 5.0
A3EFLQTQU9VYWT B000051QDD 5.0
A2E2RN093GJMKU B000051QDD 1.0
A1NEQF7O3XAHHH B000051SGS 5.0
A12SS75QOWY0QJ B000051SGS 1.0
A1Q5SM3XC4QMWR B0000520GE 4.0
A1JKZMTGY35FJ5 B0000520GG 3.0
A1EK5OVIUK6I8J B0000524NH 3.0
A11HV501OX9MNV B0000524NP 5.0
A2MQ5O07UDJDOA B0000524PB 5.0
A1TT51FSXE1V6E B000052WM0 2.0
A2UZW9I9CIZ2SN B000052WM2 1.0
A31TJWPGFOF2IH B000053F7X 4.0
A37G6UPLDGH81P B000053F92 4.0
A4ZS08A5VLP13 B000053UJ2 5.0
A1AXJJ3GIUQZ2J B000053UJ2 5.0
A1RVW523DPB1O9 B000053VWB 4.0
AS77LVLUSX18R B000053VWB 4.0
A2UVSPJPLRXAB7 B0000541B2 5.0
A243SM4D9ANHTK B0000542BR 1.0
A1MQHE3CUJN9O9 B00005444V 5.0
ABA2OLHDLUH8V B0000544GF 5.0
A3PQONZYDMXUYG B000055Y1L 1.0
A2KKJGDIWM4LWX B000055YVY 5.0
A1KWOG8QCVSGYX B00

A3RYW4G3Z39068 B00023JJUW 5.0
A3QWW7CZ6YF881 B00023XXMC 1.0
A1MQHE3CUJN9O9 B00023XXMM 5.0
ABN5K7K1TM1QA B00023XXMM 1.0
A2XLTUZCUZBIRZ B00023XXMM 5.0
A3HM5V9OZQ2X9O B00023XXMM 1.0
A3NE9POSQM0KBN B00023XXN6 5.0
AZXUFTJVXJWL0 B00023XXN6 5.0
A1SP4ZUW7AAX7Z B00023XXN6 5.0
AURU1SCV1E44T B000246VNY 1.0
A19ZIVXK12BIF0 B00024ALEO 5.0
A1WKAD7QAJUPTM B00024ALEY 5.0
AD0Y68I0JDM4Z B00024ALEY 4.0
A2NBLG314SMFLB B00024GSBO 3.0
A3GKMQFL05Z79K B00024W1RO 5.0
A37NYBB0PENCYP B00024W1TC 5.0
A16ZVU06V1XE0P B00024W1TC 4.0
AHL9IOHRJA8NZ B00024W1TC 4.0
A1PID2MT8MMPAF B00024W1U6 4.0
A3PPPKFRLJKP70 B00025LDT0 3.0
ABSYIF0ONC2AF B00025LDT0 2.0
AVHZIQ8NZ9HDC B00026D13A 4.0
A2YZHXH14W9EH1 B00026D13A 4.0
A1OUC36X8C5S49 B00026D14Y 5.0
ACMO0RPLWYRHV B00026I0GI 5.0
A17WYM7DFS6Q4Q B00026V3V2 4.0
ABN5K7K1TM1QA B0002719LK 2.0
AHNHLGKIZ2N3B B0002719LK 4.0
A2QM2I68P1TS2Q B000271N2A 2.0
A61VFLXPI073P B00027CXEM 3.0
A2U4QIPX4VCB2B B00027CXEM 3.0
A2D2A1D7IWAYRK B00027JYSU 4.0
A2PEEMUL4P3NNH B00027NWRY 5.0
A1WR6M4Z2G36O1 B00029

A3HY5ZPRUVQ2QT B000930DMI 1.0
A3VSUJZC7D352O B000930DMI 2.0
ABAZFH25304ZG B000930DO6 4.0
A1N8ST7FQO4S3K B000930DOG 3.0
A3JJ4ALE7NAHNO B000930DOQ 5.0
AZAC7B4HD5ZVC B000930DOQ 4.0
AD81H63G30R01 B000930DQ4 5.0
A03550173UTIP5TNQ9T7P B000930DQ4 5.0
A3GKMQFL05Z79K B0009350BW 4.0
A1V0HTCMZFEPX B0009350C6 3.0
A3IQINSQTRA47X B0009350C6 4.0
A3GKMQFL05Z79K B0009350CQ 4.0
A2TZH1QW0RPVQY B0009350CQ 4.0
AOJG1BGKCDHUO B0009350CQ 5.0
AC6HSBR5AAC9F B0009351Q6 2.0
AXPNCZLEHGEU7 B0009351QG 1.0
A3B6PPHSX1S17N B000935256 3.0
A3V4T1DLKVX7RM B000935256 2.0
A6MMB554OO7PQ B000935256 1.0
AEICY0E9YMZDP B00094OT4K 5.0
A3GKMQFL05Z79K B00095LEC4 5.0
A2I2HDCI4LJYNU B00095LEC4 5.0
AXSLVHRN04W23 B00095LEC4 5.0
A3TF8ZZ0S5UWTL B00095LEDS 3.0
AWBZL1Z24VMQG B00095LMJY 5.0
A38EONFW8FXOHL B00099QQF0 5.0
A6VVI484S6IN0 B00099Z304 5.0
A1IC0UJ9431PV3 B00099ZAGQ 1.0
A3Q840JGJIBAXG B0009A4EUI 5.0
A3OGTHIAGTJJ0E B0009A4EUI 5.0
A1PID2MT8MMPAF B0009A4EV2 4.0
A1I249C52EWDVP B0009A4EV2 5.0
A3GKMQFL05Z79K B0009A4EV2 4.0
A3B6PPHSX1S17N 

A3AIWMT8SI0ECY B002BXN6GY 4.0
A10DOKH5LKMWBJ B002BXN6GY 5.0
A2V2CF4OKZQMBY B002BXN6GY 5.0
AJWY5XKWLBYVM B002BY69V2 4.0
A3VOMHENYJIUMO B002BZ11E6 5.0
A2C76864JD9FDT B002BZ11E6 5.0
A20DJAE6VT8YS5 B002BZ11E6 5.0
AMNCFGWRH33DT B002C0VNQQ 3.0
AGXMLPCYBOL55 B002C0VNQQ 5.0
APB704JZPBINW B002C0VNQQ 2.0
A1IKVUK2Z8YHEY B002C0VNVG 1.0
A2KIS2T7DEDNPR B002C1TJP2 5.0
AH4GSFC9RQRY7 B002C5BMFI 1.0
A3DATE13RILKJ3 B002C5BMFI 5.0
A1GHI9BP3QCI1J B002C5BMFI 1.0
A2TYQF7XPEFZ81 B002C5BMFI 1.0
ARFD7HER7T59J B002C5BMFI 1.0
A3GDWE317FLU1D B002C5Z0SS 4.0
A20IGGGE1URYOQ B002C7RBPQ 3.0
AT7VDL3A3990C B002C7TFJQ 5.0
A30IT8OTM4YSQX B002C7TFVO 3.0
A32CWAFI1NJSPB B002CH5VIA 5.0
A25LNX54134GAV B002CIN77G 5.0
A2A8SBUHJ58LQS B002CJ8Z04 4.0
A219BE1FNRFT8T B002CJ8Z04 3.0
A1ZMJYMCK9VKAW B002CMU7CK 4.0
A24FKMQGXXOAMQ B002CMU7V6 1.0
A11FHE5HXF9P20 B002CMU89C 4.0
A3HPCRD9RX351S B002CSSEBU 3.0
A35P6OYZGBSFXR B002CT1RA4 5.0
A39KOV1WVZB2RU B002CUVXWU 2.0
A1PID2MT8MMPAF B002CZ38KA 5.0
ASZQGOOFYAL50 B002CZ38KA 5.0
A23ECZAY329OHO B00

A117XOJHW530GK B0030F1DOO 5.0
A28J44S3O90KPA B0030F1DQC 2.0
AHNT97DWEAQUY B0030F1DQC 3.0
A276BSDSBYTULH B0030GBSUC 2.0
AHEN23HPTMVYR B0030GBSUC 5.0
AW34RHIYU9RL4 B0030GBTYC 5.0
A20J0NBT6FWXXM B0030GBU28 4.0
A2JXS222CBMZP0 B0030GBU9G 4.0
A1OJ9Y6XM38ZAO B0030GBU9Q 5.0
A21MA5GFQNHC9G B0030GBUBY 4.0
AH0F0OBXF8U2V B0030GBUCS 3.0
AF05DD86VL3P1 B0030GG17C 5.0
A9XBXVQOWKHY8 B0030GG2OO 1.0
A33TLBCXHX0OLD B0030H6ZA4 5.0
A9RYS43RYO0FL B0030HIULQ 5.0
AV37OT7S682UW B0030HM2YM 5.0
AOKS5JEGN6G B0030HRNY6 4.0
AV37OT7S682UW B0030HRNY6 5.0
A20V6ZRZT4RYYP B0030MICZU 4.0
A1GSO89EHK1TZP B0030MOTNE 1.0
A2PUSX8PK0SVV3 B0030MOTNE 2.0
A2TZH1QW0RPVQY B0030MOTNE 4.0
A2AK59AGLZST3N B0030MOXA8 2.0
APNEJAYET918T B0030MQPQ8 5.0
A1H1TLRWG5OB8N B0030MQPQ8 5.0
A3HNBLO0KIJVTS B0030MQPQ8 4.0
A23G8MNSQQXFR B0030U1YYS 2.0
A1QYTCXVMPWA2E B0030YBY80 5.0
A1TTA35U4QNYG7 B00316ZC9Y 5.0
A35WERWSQE980G B003171CEW 4.0
A69TLXVT9J1TB B003171CEW 5.0
A1KX5QCQOF5FR8 B00319DX4M 4.0
A37IUGEV9L1KYN B00319DX4M 4.0
AKR6V44KXPYU1 B00319DX4M 

AYEORJS22WPQT B004CD9X2W 3.0
AKS5VW8YRUG2 B004CD9X2W 2.0
A2HMKPFXFYTSHN B004CD9X2W 5.0
A2RLL6LJM33TDD B004CD9X2W 5.0
A3TB05VA1BASLV B004CD9X2W 5.0
A36VIMWW23ETKH B004CDBP9G 3.0
A1K8JYLHRV3VGB B004CDBP9G 5.0
A21INUA35HX4XQ B004CDBP9G 1.0
AP6ZR3DA2IR78 B004CDBP9G 1.0
A3VZ8K4JZ64RN1 B004CDGG4U 4.0
AULKHDXPM8TXI B004CFASL0 3.0
AE2YO164FFGDO B004CHHZ7I 5.0
A3G05TGKE8CW25 B004CHLNWQ 5.0
A32T31MLXP3YX6 B004CUOD70 2.0
A1Q3TKFB4ZZR4Q B004CVQUOI 2.0
AYHIFI95TS76J B004CVQUOI 5.0
A3SBFK6VX93LKG B004CVQUOI 1.0
ANPKULUIW0V8S B004CVQUOI 1.0
A1VZ69YQ4WPMO5 B004CVRAFQ 5.0
A7OCU95Q73VXV B004CVWETI 5.0
A3GKMQFL05Z79K B004CVWETI 4.0
AMFIPCYDYWGVT B004CWQKDS 5.0
A3HPENYHP54IZ1 B004D05GIO 5.0
A1K0TDMCOJOUYL B004D1LKC4 5.0
A1P0OVX2637TYW B004D1SH4S 5.0
A1RI2NEFN67CRN B004DDIYP8 5.0
A3P1EY998NX589 B004DDIYP8 5.0
A1IL2AJT6RQJ3I B004DI5H26 5.0
ABNCHRDRYLTGV B004DI5H26 4.0
A1D5853Z6NWCL5 B004DI5H26 5.0
A3PICKL3USPHAG B004DPML64 5.0
AKIQOM4YNYKBV B004DTU0W2 1.0
A2U6KZ35G2P4F8 B004DTU0W2 2.0
A26F9C6KINNOZV B004DTU

A2U6KZ35G2P4F8 B00J06HWC4 5.0
A2SMBUD39PA2DV B00J06HWC4 4.0
A1T8FE0392B74W B00J226358 3.0
A2UCD2SB8LOLYG B00J4SYH3K 4.0
A131SVIMGT2IQB B00JB4WHWU 1.0
A38ZJ6EDY4YWK9 B00JJSUP18 1.0
ABPX680DGJ5AA B00JM3R6M6 5.0
A1XFP2FX9BTPGA B00JQHU9RC 5.0
A24Y1HYSNQU61H B00JQKPU0U 5.0
A3M3M1TEJTWBPT B00K4S9C9I 5.0
A364QHZHDP77LO B00KAI3KW2 1.0
A2CCPR7V6TUD31 B00KDNFG9Y 5.0
A39GZJX3Q3PPFK B00KE6DCF0 1.0
ATS9ERZJ01HUH B00KGD69U6 5.0
A3FUIYNOIAWCX B00KGGJPX6 4.0
AYHRWER9LHSQO B00L4KVCYS 5.0


In [58]:
rating_matrix_video_6k

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [59]:
unique, frequency = np.unique(rating_matrix_video_6k, 
                              return_counts = True)
print("Unique Values:", 
      unique)
  
# print frequency array
print("Frequency Values:",
      frequency)

Unique Values: [0. 1. 2. 3. 4. 5.]
Frequency Values: [83185948     1792     1109     2079     4065    10007]


## Creating rating matrix for movies and TV domain

In [60]:
names = ['user_id', 'movie_id', 'rating', 'timestamp']
df    = pd.read_csv('ratings_Movies_and_TV.csv', sep=',', names=names)
df.head()

,user_id,movie_id,rating,timestamp
0,A3R5OBKS7OM2IR,0000143502,5.0,1358380800
1,A3R5OBKS7OM2IR,0000143529,5.0,1380672000
2,AH3QC2PC1VTGP,0000143561,2.0,1216252800
3,A3LKP6WPMP9UKX,0000143588,5.0,1236902400
4,AVIY68KEPQ5ZD,0000143588,5.0,1232236800


In [61]:
df = df.loc[df['user_id'].isin(new_user_ids_mt_6k.tolist())] 
df.shape

(29038, 4)

In [62]:
new_item_ids_mt_6k = df.movie_id.unique()
print(new_item_ids_mt_6k.shape)
print(new_item_ids_mt_6k)

(14972,)
['0005019281' '0307142469' '0307142485' ... 'B00KHA99BY' 'B00KM9LY3Q'
 'B00L4IDS4W']


In [63]:
print(item_ids_mt.shape)

(200941,)


In [64]:
item_ids_mt_6k_rated_unrated = new_item_ids_mt_6k
np.random.shuffle(item_ids_mt_6k_rated_unrated)
print(item_ids_mt_6k_rated_unrated.shape)

(14972,)


In [65]:
rating_matrix_mt_6k = np.zeros([7000, item_ids_mt_6k_rated_unrated.shape[0]], dtype='float32')
print(rating_matrix_mt_6k.shape)

(7000, 14972)


In [66]:
for ind in df.index:
    if df['user_id'][ind] in new_user_ids_mt_6k:
        rating_matrix_mt_6k[np.where(new_user_ids_mt_6k == df['user_id'][ind])[0][0]][np.where(item_ids_mt_6k_rated_unrated == df['movie_id'][ind])[0][0]] = df['rating'][ind]
        print(str(df['user_id'][ind]) + " " + str(df['movie_id'][ind]) + " " + str(df['rating'][ind]))

A2S8D0QSJZ07DJ 0005019281 5.0
ATXWVGLKWDPX4 0005019281 5.0
A3VERR8USXCHS4 0005019281 5.0
AIMVNK843DDSK 0005019281 4.0
A2B8QMAW3RLAB1 0307142469 4.0
A26U24D5UYEVW2 0307142485 5.0
A2ZBHG85BL6K3C 0307142493 5.0
A1OBPHRXHZF8P6 0307142493 5.0
A1OBPHRXHZF8P6 0307514161 5.0
A1FWN4IK6CB0Q7 0307889718 5.0
A55MRYPUAX4QU 0310263662 5.0
A2P2B90RR2A082 0310263662 4.0
A16ZFFYQ98RGX2 0310263662 4.0
A1POGYGC5C99W5 0310263662 3.0
A23Z6QIBPNCO28 0310263662 5.0
AWXM455BNL2RM 0310263662 3.0
A1VU8D2148AKAG 0310263662 4.0
ATS1HA5M3EPZ4 0310263662 5.0
A5H8VLE7L7SB 0310263662 5.0
A36Q1RC3P3OJHH 0310263662 5.0
A258OVP9ZCZHFV 0310263662 3.0
A3FMCN2FJPAJ8A 0310263662 1.0
A17FLA8HQOFVIG 0310263662 4.0
A1XU2VGHMZEHG4 0310263662 4.0
A3NM0RAYSL6PA8 0310263662 5.0
A4QC76NMWE5OC 0310263662 5.0
A28D1YJ1NY60CI 0310263662 5.0
A15AZ0IHFYHE0R 0310263662 5.0
A32RITC3YVWNPH 0310263662 5.0
A2AOCMTCD0JAAT 0310263662 5.0
AYADB9DWINJID 0310705657 5.0
A1PCJ32AXQWLI1 0321680138 5.0
A1E319EHBAD5VK 0328048216 5.0
A2QU1LKHVYQSGN 0615

A2WNPTIY4DQ3E3 0780628535 1.0
A2OPY0ZQSYM13C 0780628535 4.0
A2WNPTIY4DQ3E3 0780628551 2.0
A2EAI4948TW53X 078062856X 5.0
A15Q9YEG1XPEJN 078062856X 4.0
A9A3LVQRH41IY 078062856X 5.0
A1QF10TB8AYTA3 078062856X 4.0
A2Z2W192L0WN9B 0780628799 3.0
A1IJ239N9ZOPYH 0780628799 5.0
A2PBYPQDGE5LFV 0780628799 5.0
A1R98YOH1ZQIGJ 0780628799 5.0
A17FLA8HQOFVIG 0780628799 5.0
A329TTW9VJ2NBU 0780628799 4.0
ACMOLKXVKQ06Z 0780628799 5.0
A3F4OHY3R93F78 0780628799 5.0
A2ZWDXG11V4LXF 0780629248 5.0
ABN5K7K1TM1QA 0780629450 5.0
AY36D1JZT87K2 0780629450 5.0
A2AEIFKWEW2J4W 0780629450 5.0
A38TC59GM2WW1X 0780629450 5.0
A2HMB1IEBSGSHJ 0780630106 5.0
A3JEJ8WE3VCOCK 0780630459 5.0
A1NZNSH0B3TT7Q 0780630858 1.0
A1EQ0W58BZO9A0 0780630858 1.0
A1EQ0W58BZO9A0 0780630866 5.0
ADHNXAGJWB4O6 0780630874 3.0
A1OBPHRXHZF8P6 0780630882 1.0
A1NZNSH0B3TT7Q 0780630882 4.0
A1EQ0W58BZO9A0 0780630890 2.0
A3BYMKUJZL2XNY 0780630904 2.0
A2A7CI7OWBOI3A 0780630904 3.0
A1BOJ7P7IEWGBT 0780631153 4.0
A15Q9YEG1XPEJN 0780631153 1.0
A1RVW523DPB1O9 

A3A79B8FRJ64UV 0788860704 5.0
A1GR7YQLBT8N83 0788860704 5.0
A1I7QGUDP043DG 0788860704 5.0
A2YDBDW866CW85 0788860704 5.0
AFW7TX0B0IXY3 0788860704 5.0
A27ZOCD5B63Y0P 0788860704 5.0
A15Q9YEG1XPEJN 0788860704 4.0
A38UDGZIKQCK3E 0788860704 5.0
A3R2UDSGPLHF3Y 0788860704 5.0
A1W9TMK21D5GZI 0788860704 5.0
A2QHJROLDZDL2S 0788860704 5.0
AZDHVWXIHV2FJ 0788860704 5.0
A3P9D22KDEXVMN 0788861948 5.0
A17FLA8HQOFVIG 0788861948 5.0
A2BZ7MYTSNYMEW 0788861948 5.0
AZAC7B4HD5ZVC 0788861948 4.0
A17FLA8HQOFVIG 0788867164 5.0
ASTUGXCVUM8L 0788868446 5.0
A3F38EELK8ZGXA 0788868446 5.0
A1I7QGUDP043DG 0788869973 5.0
A1PE07FDHM05I7 0788882988 5.0
A1I7QGUDP043DG 0788882988 4.0
ABN5K7K1TM1QA 0788882988 4.0
A3UHWNH1PZYEUJ 0788882988 3.0
A3P9D22KDEXVMN 0788882988 4.0
A34HL52AN3IZ50 0788882988 5.0
A17FLA8HQOFVIG 0788882988 3.0
A3NM0RAYSL6PA8 0788882988 2.0
A2BZ7MYTSNYMEW 0788882988 4.0
A2MWXZP6VK8I8S 0788882988 1.0
A2OMUPIEGQ8YZI 0788882988 1.0
A1UEDQYDO58T69 0788882988 1.0
A1TH45SI84KQJW 078943783X 5.0
A1I7QGUDP043DG 0

A15Q9YEG1XPEJN 6300182320 2.0
A3D6DPWQW509R9 6300182320 5.0
AZ294IDD2XGXE 6300182320 5.0
A11HZS5E55U6TH 6300182320 5.0
A15Q9YEG1XPEJN 6300182355 4.0
AZ294IDD2XGXE 6300182355 5.0
A2LP9JP1TLK65K 6300182355 5.0
A1BDGJQWCFQBYJ 630018241X 5.0
A1WKTQMHQVLNA9 630018241X 5.0
A2CZPM110DW516 6300182495 3.0
A17FLA8HQOFVIG 6300182517 5.0
AURSFTX4YP96N 6300182541 5.0
ABN5K7K1TM1QA 6300182711 4.0
A2WB4OWBUH2VQX 6300182738 5.0
A3EQ6BM8LI7YB2 6300182762 3.0
A2AMX0AJ2BUDNV 6300182762 5.0
A3GKPMHV4U64IY 6300182800 4.0
A15Q9YEG1XPEJN 6300182851 4.0
A1NCZ6O4GP210A 6300182908 5.0
A37KO6OVBPLDA5 6300183009 5.0
A1PHHO49YEDEY4 6300183092 5.0
A2CZPM110DW516 6300183092 5.0
AP3SM2STURGQY 6300183181 3.0
A39O9MBHEGDBVO 6300183467 5.0
AWXM455BNL2RM 6300183483 3.0
ABN5K7K1TM1QA 6300183513 4.0
A2OMVUWP0E0RB9 6300183513 5.0
A1NANM7VNUUXDK 6300183513 5.0
A29R4FCO6RFX4K 6300183513 5.0
A17FLA8HQOFVIG 6300183513 4.0
A3NM0RAYSL6PA8 6300183580 3.0
AJLXEIMFA80CY 6300183653 5.0
A37NYBB0PENCYP 630018370X 5.0
A3281EUZKOO7UO 630

AZAC7B4HD5ZVC 6300248143 4.0
A1QBDRLWNOIL5L 6300248178 3.0
A3GKMQFL05Z79K 6300248178 4.0
AT2BYFEVLVITT 6300248178 5.0
A2YIN60Q7T7M2D 6300248178 5.0
AJLYAY1G5Z8GE 6300248178 5.0
A29R4FCO6RFX4K 6300248178 4.0
A1Y19SOGNP3CQO 6300248178 5.0
ATWBOP6K37WP2 6300248569 4.0
A2VWE5SGNDS8HW 6300248569 3.0
A1I8SSGKSIBWVE 6300249824 4.0
AUZF6JFWU6LLS 6300249824 3.0
A1I7QGUDP043DG 6300249964 4.0
ABN5K7K1TM1QA 6300249964 3.0
A1FCTXPRHBXNNO 6300249964 4.0
A25YHAWCM3WLTU 6300250172 1.0
A92LD431P843F 6300250512 5.0
A2ZWE0C4N5ZP8 6300250512 4.0
A7KVKT0XFJSWI 6300250776 5.0
A27ZOCD5B63Y0P 6300251004 5.0
AKLCOJW6JYFBW 6300251004 4.0
A1M6DY7Y1W5AVP 6300251004 5.0
ASIIGDWFMU46J 6300251004 5.0
A3CTF1BWZQ1THN 6300251004 5.0
AEWW0COS1TEDN 6300251004 4.0
A1VTC39L2NAZ8U 6300251004 5.0
A2I0X28L5E42VU 6300251012 5.0
A1EA9VGXN2IWUJ 6300251012 5.0
A35W8YZ6K4AOFV 6300251020 1.0
A17FLA8HQOFVIG 6300251217 4.0
A4L0UD2AWHART 6300251217 5.0
A28RBLQ0ZX4HP7 6300251330 4.0
A2MIX9TG8SR9BT 6300251330 5.0
A9ADYTHI8MX3M 630025161

...

AA27FNAQ8HVYG B0056486RE 5.0
A3VOMHENYJIUMO B0056DH1IU 5.0
AFPNQLIEIMY2F B0056G1AM0 5.0
A34HL52AN3IZ50 B0056KOCDA 5.0
AV37OT7S682UW B0056KOCDA 3.0
A3GKMQFL05Z79K B0056OZHYE 4.0
A3GKMQFL05Z79K B0056P6SOG 3.0
AWQPN8016AO06 B0056P6SOG 1.0
AZ16IZGQ3WLTZ B0056YMDW8 5.0
AYFXM5IV65UYB B0056YMDW8 5.0
A2YWNXD2JYRH9U B0056YMDX2 5.0
A1816S7T5W2CDK B0056ZRP02 5.0
AGNZ3W73LG0CU B00573QVNA 4.0
A3O4MNHASO099T B00576U9AS 5.0
A31OGMI9Z40EMM B00576U9GC 5.0
A3IE4W5CGXNN5S B00576U9GC 5.0
A3F4OHY3R93F78 B00576U9LW 4.0
A3QR1O3JXNK7UA B0057ACHFO 4.0
A34HL52AN3IZ50 B0057FGD1I 1.0
A2MJCWXPTWKV1Y B0057J5BMG 5.0
A3EJKPB8J0PL5I B0057J5BMG 5.0
A1FENS8DSOET1G B0057LOEGS 4.0
A23T4J4BRHTNE6 B0057O6IG4 3.0
A3KQNW60X7UXEH B0057OPOZA 4.0
A10X3QD6WG6EIJ B0057OQNCI 5.0
A1MVE782GW01SF B0057P4SCY 2.0
A1FN2XDPBWR4JF B00584A7MY 4.0
AS0U43OBIEXQL B0058O1FHK 4.0
AVH28SA2CH54N B0058TNSGG 5.0
A2AFTRU43PY9P5 B0058TNSGG 5.0
A2DYC7PTNNMANE B0058YPG1G 5.0
AXZ5HPL5P53J0 B0058YPG1G 5.0
A1UD8NKXB3868M B0058YPGSY 5.0
ATXWVGLKWDPX4 B0058Y

A24V6I409AQDCV B008NA3I1W 5.0
A2H9R97DAQLJ77 B008NNY8M2 2.0
A3HNBLO0KIJVTS B008NNY8M2 4.0
A3F4OHY3R93F78 B008NNY9HQ 5.0
A1TT9TJI3AXWPF B008NNY9JY 5.0
A39IXBGZWTUE6Q B008NNY9YE 5.0
AZ505YQE7561U B008NNY9YE 5.0
AM8BH6W8BVPKS B008NR90HQ 5.0
AVCG0GMYNF7E8 B008O5CPWE 4.0
A1VL5O955BYRJK B008O69T18 5.0
A1QMNDD5PV7L9B B008O69T18 5.0
A19XZWL9U51TAP B008OGIPJA 5.0
A3CTO7AK1PDZJP B008OGIPJA 5.0
A2CJN3IDH2DWT5 B008OGIPJA 4.0
A3QDA1L5EBVZVI B008OGIPJA 4.0
A3SRDW0GVQYPC4 B008OGIPJA 5.0
ATOMYGMSU90LF B008OGIPJA 4.0
A1EW4ZBFA054C9 B008OTTU2S 5.0
A3PZ5XXS4SRA3 B008P6168Q 5.0
A9NLS0O46SMTM B008P9M614 5.0
AGLP82TL0UAYC B008PEHZXI 3.0
AJL5VTFM8ZP3C B008PEHZXI 5.0
A1ONVQZM1S0J1C B008PEHZXI 4.0
A2P8SDVVXZ1FWQ B008PEHZXI 5.0
A2CUP8G1P2H44P B008PEHZXI 5.0
A3EUC19RRVLUZE B008PEHZXI 2.0
A1ACW120D309LH B008PEHZXI 4.0
A2MSJ2NV99JXQC B008PEHZXI 5.0
A1GUNZMIC8AHPY B008PEHZXI 5.0
ASC0HLBZE8327 B008PEHZXI 3.0
A1M9AA4UM74690 B008PEHZXI 5.0
A10B7PD3UVSG3C B008POPRS8 1.0
A3H43YTXXTHCI3 B008POSU3W 5.0
A3GKMQFL05Z79K B008

A3GKMQFL05Z79K B009B8YZHQ 5.0
A23ZUK1J474V80 B009B8YZIK 5.0
A2NTO0WI0E4KQL B009B8YZIK 5.0
A2VMW474M9LB77 B009B8YZIK 4.0
A3FKAP9OJG07X B009B8YZIK 5.0
AKQDQAC1ITJ3R B009B8YZIK 4.0
A15IFSOBJXFKBH B009B8YZIK 5.0
A1WSWGB24SHX9L B009B8YZIK 2.0
A2QOOIE73FJIVA B009D0AZIK 5.0
ATBGV8L0LFS3X B009D1G9S4 4.0
A37IHRVO8JWYF2 B009D4SFEC 5.0
A343NQGW9SQB8V B009DH371K 5.0
AD1H62BLZDBOH B009DH378I 2.0
A2GL66ZKGIMVR0 B009DJKKKE 5.0
A30T8OM11F3M0J B009DRFCL8 5.0
AGLP82TL0UAYC B009DS5OEM 3.0
A3T9PA1L9D6GL5 B009DT98RK 5.0
A4PRDAPSW8BQH B009DW5YA2 4.0
A3M6RVUVZOQZLB B009EU6F1U 2.0
A2II40EJQ3C2D1 B009EU6F1U 5.0
A3G0A221KOF80I B009FB3W9G 1.0
A33GV64OPZ4PC2 B009FB3ZAW 1.0
A312MFKWEGW6PV B009FFHWRU 5.0
A2U7GIXU6B5PDQ B009GGWWCS 5.0
A5OG0BU8MKWF6 B009GPH3Q4 4.0
A3VERR8USXCHS4 B009GYTOZ8 5.0
A2JPISF2OXK1GY B009GYTP3O 5.0
A1N89VQWSGZ70Z B009H3LN8Y 5.0
A2ZA9MYCJTO44C B009H3LN8Y 4.0
A2UHER42224S1W B009H3LN8Y 5.0
A1IIZT193OAT0M B009HWEIUK 2.0
A1OBPHRXHZF8P6 B009INAND4 5.0
A1Y19SOGNP3CQO B009IRWF4A 2.0
A1UQP7RI6YYH8R B0

A3GKMQFL05Z79K B00CUDD0J8 4.0
AEPMR6F2BNAFY B00CVB6UXW 5.0
AMFIPCYDYWGVT B00CWM58WY 4.0
A2B0WD72NXO0F7 B00CWM58WY 4.0
AKS2FR9SVXWUY B00CWM58WY 5.0
A37XMY9B4DSJC9 B00CWM58WY 5.0
AYFXM5IV65UYB B00CWM58WY 5.0
A39KVX7GSNZICH B00CWM58WY 5.0
AATPJIKKR8ASA B00CWM58WY 5.0
AYQ7SFRDEKB5I B00CWM58WY 5.0
A3F4OHY3R93F78 B00CXRTH8Y 5.0
A3BGEV4W5SX2M B00CXXVCBI 2.0
A3TUEDM2RFQBHQ B00CYQXBSQ 5.0
AFG91MZXHE1ZH B00CYQXFP0 4.0
AL5TQX7VV6BJ5 B00CYQXFP0 4.0
A2828G91GGRYN5 B00CYQXGMW 5.0
A2ZA9MYCJTO44C B00CZB9BCU 5.0
A38CQ67HGZ3IB4 B00D1D45R2 5.0
A2UHYTIBE6I8ZM B00D2D1WVI 5.0
AGAOZ53UU82G7 B00D2UMHWO 5.0
A1ZHOPK6UMXXBC B00D2UMHWO 5.0
A15RS0FQNF9RSR B00D2UMI9Q 3.0
A390HK0GC01AXF B00D3LA6YI 5.0
A6GUGB25MLAG7 B00D3UOE7O 5.0
A3MRHUNU5P67D B00D3XMA9A 2.0
AXYX89YWHFHD B00D44IQBY 5.0
A3F4OHY3R93F78 B00D44IQBY 5.0
A38GNSWJWZM66W B00D49YEHO 5.0
A1V2542L08YPYC B00D4C5GXC 4.0
A55QNEAJ7QCQX B00D6I7EUM 1.0
A16EDYPFM4WAXF B00D7AM2WO 3.0
A1248JRIINWRTH B00D7AM2YW 5.0
A15Q9YEG1XPEJN B00D7AM2YW 4.0
A46KQGP4ITB1R B00D7AM2YW 

A1NZNSH0B3TT7Q B00HRYH67M 5.0
A2MI35K36A3BZE B00HRYH7BC 5.0
AFHPJY26IU1ZJ B00HSTMKFO 1.0
A3EFG5MS4UKNKQ B00HTRJYAY 5.0
A138TTA1M4YOG8 B00HUCF420 5.0
A9ZFNLMFF1U8M B00HUCF7AO 5.0
A312LBXMKEOZS6 B00HUCF7AO 4.0
A2SNYK7O4PLNG1 B00HW3EJ6Y 4.0
A2EX1E7P969W5E B00HW3EJQE 5.0
A3HNBLO0KIJVTS B00HW3EZXQ 4.0
A1TTQTEH4UHV8W B00HWGT56U 5.0
A3KB3WWMA35AQ4 B00HX0KIAM 5.0
A2PVQOXIZBW0S8 B00HX0KIAM 4.0
AFF7F0BD7RO8V B00HX7EIAQ 5.0
A2DS2LVYY0RF5I B00HZMZ240 5.0
A129D6C583RMXH B00I0CAVV8 3.0
AGCEMOS8DGOQ2 B00I0VTP84 5.0
A22NB5IEE054MS B00I0VTP84 5.0
A22I4XH2VYPDQU B00I3C1T6G 3.0
A2WSL6HS4B6WHN B00I4583Y8 5.0
A3T9PA1L9D6GL5 B00I5NKP7W 5.0
A3DVYJGU3SIKLC B00I86MQF0 2.0
A3J9TDX9EDX83N B00IBKXQF2 1.0
A1RPLXNFRJA3QX B00IBKXRX8 5.0
A37XMY9B4DSJC9 B00ICWO0ZY 5.0
A2EX1E7P969W5E B00IDRQ3JO 2.0
A3P0P1P8P0W5CE B00IEA3KBO 3.0
A1NHJ5O8PGO5PH B00IGVJH0E 4.0
A195MHJJS8F7UO B00IGVJH0E 5.0
A23ZUK1J474V80 B00II4DYHG 5.0
A2BZ7MYTSNYMEW B00IJS2OHM 4.0
A9NCVAXA0PM9I B00IJZMZW4 2.0
A3KB3WWMA35AQ4 B00IKM5L6I 1.0
A2EX1E7P969W5E 

In [67]:
unique, frequency = np.unique(rating_matrix_mt_6k, 
                              return_counts = True)
print("Unique Values:", 
      unique)
  
# print frequency array
print("Frequency Values:",
      frequency)

Unique Values: [0. 1. 2. 3. 4. 5.]
Frequency Values: [104774962      1953      1628      3020      6159     16278]


In [68]:
numpy.savetxt("rating_matrix_video_6k.csv", rating_matrix_video_6k, delimiter = ",")

In [69]:
temp = pd.DataFrame(new_user_ids_video_6k)
temp.to_csv("new_user_ids_video_6k.csv")

In [70]:
temp = pd.DataFrame(item_ids_video_6k_rated_unrated)
temp.to_csv("item_ids_video_6k_rated_unrated.csv")

In [71]:
numpy.savetxt("rating_matrix_mt_6k.csv", rating_matrix_mt_6k, delimiter = ",")

In [72]:
temp = pd.DataFrame(new_user_ids_mt_6k)
temp.to_csv("new_user_ids_mt_6k.csv")

In [73]:
temp = pd.DataFrame(item_ids_mt_6k_rated_unrated)
temp.to_csv("item_ids_mt_6k_rated_unrated.csv")

## Removing overlapping test user_ids from target domain

In [214]:
d                        = 100 

In [215]:
overlapping_user_ids_t = np.intersect1d(new_user_ids_video_6k, new_user_ids_mt_6k)
print(overlapping_user_ids_t)
print(overlapping_user_ids_t.shape)

['A01632683TJ1GCADQ8B7X' 'A031533437UL5KXSH8FNB' 'A03550173UTIP5TNQ9T7P'
 ... 'AZWM4WE39FNY3' 'AZXUFTJVXJWL0' 'AZZOUDTWW6ZYZ']
(6000,)


In [216]:
np.random.seed(13)
train = np.random.choice(overlapping_user_ids_t, size=math.ceil(0.8*overlapping_user_ids_t.shape[0]), replace=False)
test = np.array(list(set(overlapping_user_ids_t.tolist()).difference(set(train.tolist()))))

In [217]:
X_train = np.empty((0,100))
Y_train = np.empty((0,100))
X_test = np.empty((0,100))
Y_test = []

In [218]:
for user_id in test:
    ind1 = np.where(new_user_ids_mt_6k == user_id)
    Y_test.append(rating_matrix_mt_6k[ind1][0])
    new_user_ids_mt_6k =np.delete(new_user_ids_mt_6k, ind1, 0)
    rating_matrix_mt_6k = np.delete(rating_matrix_mt_6k, ind1, 0)

In [219]:
Y_test = np.array(Y_test)
print(Y_test.shape)

(1200, 14972)


In [220]:
print(new_user_ids_mt_6k.shape)
print(rating_matrix_mt_6k.shape)

(5800,)
(5800, 14972)


## GRS for Video Games domain

In [46]:
epochs                   = 3
tstPer                   = 30                               # Testing Percentage
NoTstIFromG              = 50
l                        = 5                                # Rating Level

K1                       = 10                              #Number of Clusters in User Space
K2                       = 22                              #Number of Clusters in Item Space

d                        = 100                              # latent space size
minUserPerForSel         = 20                               #Item will be select for test only if minUserPerForSel percentage of users in a group has rated
k                        = 40                               # eval@k: precision@k, recall@k
cutoff                   = 3                                # Relevant > cutoff

lambda1UnifiedLS         = 1
lambda3UnifiedLS         = 1

c2                       = 10**-2
tol                      = 10**-3;
maxiter                  = 500;

In [47]:
n, m = rating_matrix_video_6k.shape
print(n, m)

9000 9245


In [48]:
#Spectral Clustering
simU                     = userSim(rating_matrix_video_6k)
simU                     = (simU + simU.T)/2
UGidx_video              = spectralClu(simU,K1).reshape(-1,1)

simI                     = userSim(rating_matrix_video_6k.T)
simI                     = (simI + simI.T)/2
VGidx_video              = spectralClu(simI,K2).reshape(-1,1)

In userSim
In userSim


In [49]:
print(UGidx_video)
print(UGidx_video.shape)

[[0]
 [2]
 [0]
 ...
 [0]
 [2]
 [1]]
(9000, 1)


In [50]:
print(VGidx_video)
print(VGidx_video.shape)

[[16]
 [16]
 [ 1]
 ...
 [ 8]
 [ 4]
 [16]]
(9245, 1)


In [51]:
unique, frequency = np.unique(UGidx_video, 
                              return_counts = True)
print("Unique Values:", 
      unique)
  
# print frequency array
print("Frequency Values:",
      frequency)

print()

unique, frequency = np.unique(VGidx_video, 
                              return_counts = True)
print("Unique Values:", 
      unique)
  
# print frequency array
print("Frequency Values:",
      frequency)

Unique Values: [0 1 2 3 4 5 6 7 8 9]
Frequency Values: [1396 1085 1540  795  375 1219  396  986  829  379]

Unique Values: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21]
Frequency Values: [148 503 506 625 263 122 456 674 453 487 283 655 618 428 448 596 635 310
 324 234 250 227]


### Recommendation

In [52]:
np.random.seed(0)
vini = np.random.randn((n*d + m*d + K1*d + K2*d), 1)

In [53]:
# Group Recommendation: Unified LS
objGrad             = gradUnifiedLS
v,numiter,ogcalls,J = conjgrad(vini,objGrad,c2,tol,maxiter,l,d,rating_matrix_video_6k,lambda1UnifiedLS,lambda3UnifiedLS,K1,K2,UGidx_video,VGidx_video)
U_video                   = v[0:n*d].reshape(n,d,order='F')
V_video                   = v[n*d:n*d+m*d].reshape(m,d,order='F')
UG_video                  = v[n*d+m*d:n*d+m*d+K1*d].reshape(K1,d,order='F')
X                   = U_video@V_video.T
X[X<=1]             = 1
X[X>=5]             = 5
XPerA               = aggregratePrediction(X,UGidx_video)
XPerAMat            = XPerA[UGidx_video.reshape(-1,),:]

XPer_Grp                    = UG_video@V_video.T
XPer_GrpMat                 = XPer_Grp[UGidx_video.reshape(-1,),:]
XPer_GrpMat[XPer_GrpMat<=1] = 1
XPer_GrpMat[XPer_GrpMat>=5] = 5


 3813789.5514
1

 2607253.2114
2

 2030770.6985
3

 1826957.3506
4

 1761249.4810
5

 1731451.5626
6

 1692631.3334
7

 1611130.9786
8

 1433019.9405
9

 1033260.4111
10

 700065.9107
11

 549807.4013
12

 427852.2552
13

 388224.9629
14

 361754.6105
15

 342236.2299
16

 326817.5642
17

 307556.8679
18

 293497.4361
19

 276679.9160
20

 258348.9152
21

 249828.5751
22

 239352.6630
23

 231545.1662
24

 207004.5904
25

 172032.9312
26

 128691.8785
27

 115574.7117
28

 109676.5592
29

 104499.0499
30

 101776.7144
31

 98443.1612
32

 95155.7594
33

 91001.8642
34

 86396.7162
35

 84735.1399
36

 82251.5082
37

 79064.3313
38

 76668.2670
39

 75072.8359
40

 73849.7003
41

 71976.1908
42

 70473.7218
43

 67198.7557
44

 65929.4032
45

 64701.1660
46

 63400.5537
47

 62487.7150
48

 61714.9726
49

 60968.7660
50

 60353.0909
51

 59821.5029
52

 59487.1933
53

 59090.9407
54

 58753.4511
55

 58415.3577
56

 58128.0802
57

 57941.8111
58

 57702.5702
59

 57474.2284
60

 57305.

In [54]:
mae,rmse,auc,precision,recall,f1,ndcg = EvaluationAllUpdated(XPer_GrpMat, rating_matrix_video_6k, k, cutoff)
ResultTrnULS_Grp                      = ["Unified_LS_Grp",mae,rmse,auc,precision,recall,f1,ndcg]
Result_Trn.loc[len(Result_Trn)]       = ["Video Games domain - Run 1",mae,rmse,auc,precision[0][0],precision[0][4],precision[0][9],
                                         precision[0][19],precision[0][29],precision[0][39],recall[0][0],recall[0][4],
                                         recall[0][9],recall[0][19],recall[0][29],recall[0][39],f1[0][0],f1[0][4],f1[0][9],
                                         f1[0][19],f1[0][29],f1[0][39],ndcg[0][0],ndcg[0][4],ndcg[0][9],ndcg[0][19],
                                         ndcg[0][29],ndcg[0][39]]

## GRS for Movies and TV domain

In [221]:
epochs                   = 3
tstPer                   = 30                               # Testing Percentage
NoTstIFromG              = 50
l                        = 5                                # Rating Level

K1                       = 10                               #Number of Clusters in User Space
K2                       = 100                             #Number of Clusters in Item Space

d                        = 100                              # latent space size
minUserPerForSel         = 20                               #Item will be select for test only if minUserPerForSel percentage of users in a group has rated
k                        = 40                               # eval@k: precision@k, recall@k
cutoff                   = 3                                # Relevant > cutoff

lambda1UnifiedLS         = 30
lambda3UnifiedLS         = 1

c2                       = 10**-2
tol                      = 10**-3;
maxiter                  = 500;

In [222]:
n, m = rating_matrix_mt_6k.shape
print(n, m)

5800 14972


In [223]:
#Spectral Clustering
simU                     = userSim(rating_matrix_mt_6k)
simU                     = (simU + simU.T)/2

UGidx_mt                    = spectralClu(simU,K1).reshape(-1,1)

simI                     = userSim(rating_matrix_mt_6k.T)
simI                     = (simI + simI.T)/2

VGidx_mt                    = spectralClu(simI,K2).reshape(-1,1)

In userSim
In userSim


C:\Users\NIT\AppData\Local\Temp\ipykernel_30256\3829436230.py:21: RuntimeWarning: invalid value encountered in true_divide
  B1               = nominator/denominator


In [224]:
print(UGidx_mt)
print(UGidx_mt.shape)

[[0]
 [0]
 [0]
 ...
 [1]
 [0]
 [0]]
(5800, 1)


In [225]:
print(VGidx_mt)
print(VGidx_mt.shape)

[[58]
 [51]
 [43]
 ...
 [57]
 [49]
 [78]]
(14972, 1)


In [226]:
unique, frequency = np.unique(UGidx_mt, 
                              return_counts = True)
print("Unique Values:", 
      unique)
  
# print frequency array
print("Frequency Values:",
      frequency)

print()

unique, frequency = np.unique(VGidx_mt, 
                              return_counts = True)
print("Unique Values:", 
      unique)
  
# print frequency array
print("Frequency Values:",
      frequency)

Unique Values: [0 1 2 3 4 5 6 7 8 9]
Frequency Values: [4125  191  164  194  184  197  176  181  218  170]

Unique Values: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98 99]
Frequency Values: [279 248 327 154  68 268 281  17 247 204 393 168   6 108   4   7  79 226
 272 337 285 342 144   5   6 239 381   3 120 229 288  11 179 360 258 392
 259   9  35   6   2 103 251 271 113 248 221 259   4 151  28 346   1  15
   3   2   5 450 434  10   4 188  16   2   6  16 351 131   4 383 352  21
  77 268 363   2   2   2 424   7  10 163 210 177 176   1   2 224  56 288
   9   2   7   1 212 102 395  19   1 137]


### Recommendation

In [227]:
np.random.seed(0)
vini = np.random.randn((n*d + m*d + K1*d + K2*d), 1)

In [228]:
# Group Recommendation: Unified LS
objGrad             = gradUnifiedLS
v,numiter,ogcalls,J = conjgrad(vini,objGrad,c2,tol,maxiter,l,d,rating_matrix_mt_6k,lambda1UnifiedLS,lambda3UnifiedLS,K1,K2,UGidx_mt,VGidx_mt)
U_mt                  = v[0:n*d].reshape(n,d,order='F')
V_mt                   = v[n*d:n*d+m*d].reshape(m,d,order='F')
UG_mt                  = v[n*d+m*d:n*d+m*d+K1*d].reshape(K1,d,order='F')
X                   = U_mt@V_mt.T
X[X<=1]             = 1
X[X>=5]             = 5
XPerA               = aggregratePrediction(X,UGidx_mt)
XPerAMat_mt            = XPerA[UGidx_mt.reshape(-1,),:]

XPer_Grp                    = UG_mt@V_mt.T
XPer_GrpMat_mt                 = XPer_Grp[UGidx_mt.reshape(-1,),:]
XPer_GrpMat_mt[XPer_GrpMat_mt<=1] = 1
XPer_GrpMat_mt[XPer_GrpMat_mt>=5] = 5


 64488444.9544
1

 54958145.0034
2

 35711693.3217
3

 33392332.4413
4

 32374251.9575
5

 30750394.2146
6

 27614155.4620
7

 27207447.1827
8

 23109458.5052
9

 19100843.1124
10

 18440890.7226
11

 15005695.3670
12

 13727306.6112
13

 12082011.7263
14

 10926873.5435
15

 10712061.5935
16

 9615855.6682
17

 8705901.7836
18

 8123670.4296
19

 7877920.4157
20

 7119254.3598
21

 6570168.2046
22

 6514470.9778
23

 6095404.7370
24

 5530054.1802
25

 5423927.1683
26

 4481570.3411
27

 3864737.1003
28

 3690109.9202
29

 3036561.3567
30

 2957105.1608
31

 2443149.9694
32

 2011398.1605
33

 1903802.5546
34

 1555460.4086
35

 1494695.8182
36

 1256807.5496
37

 1052822.8407
38

 1022970.7042
39

 853701.6198
40

 793180.4012
41

 688748.0706
42

 583891.6426
43

 575376.0075
44

 520090.0999
45

 494892.6487
46

 477563.4434
47

 438206.4284
48

 434179.3780
49

 403487.0435
50

 389601.2119
51

 377423.8781
52

 368432.6817
53

 366863.7294
54

 356752.3857
55

 344799.7548
56

 

In [197]:
# Result_Trn.drop(index=Result_Trn.index[-1],axis=0,inplace=True)

In [229]:
mae,rmse,auc,precision,recall,f1,ndcg = EvaluationAllUpdated(XPer_GrpMat_mt, rating_matrix_mt_6k, k, cutoff)
ResultTrnULS_Grp                      = ["Unified_LS_Grp_mt",mae,rmse,auc,precision,recall,f1,ndcg]
Result_Trn.loc[len(Result_Trn)]       = ["Movies & TV domain - Run 3",mae,rmse,auc,precision[0][0],precision[0][4],precision[0][9],
                                         precision[0][19],precision[0][29],precision[0][39],recall[0][0],recall[0][4],
                                         recall[0][9],recall[0][19],recall[0][29],recall[0][39],f1[0][0],f1[0][4],f1[0][9],
                                         f1[0][19],f1[0][29],f1[0][39],ndcg[0][0],ndcg[0][4],ndcg[0][9],ndcg[0][19],
                                         ndcg[0][29],ndcg[0][39]]

In [230]:
Result_Trn

,Model,MAE,RMSE,AUC,Precision@1,Precision@5,Precision@10,Precision@20,Precision@30,Precision@40,...,F1@10,F1@20,F1@30,F1@40,NDCG@1,NDCG@5,NDCG@10,NDCG@20,NDCG@30,NDCG@40
0,Video Games domain - Run 1,1.293497,1.492651,0.833634,0.854556,0.832072,0.829426,0.828472,0.828302,0.828198,...,0.845354,0.846971,0.847404,0.847510,0.918097,0.985517,0.992986,0.995109,0.995601,0.995781
1,Movies & TV domain - Run 1,1.254993,1.403518,0.795579,0.900517,0.881057,0.876802,0.874823,0.874273,0.873867,...,0.886542,0.892339,0.894021,0.894639,0.862662,0.965177,0.981814,0.989506,0.991778,0.992798
2,Movies & TV domain - Run 2,1.307721,1.450869,0.794652,0.905345,0.883796,0.879746,0.878013,0.877169,0.876773,...,0.888987,0.895234,0.896769,0.897453,0.861628,0.964727,0.981462,0.989088,0.991646,0.992665
3,Movies & TV domain - Run 3,1.038399,1.326876,0.831459,0.911897,0.882718,0.877693,0.875657,0.875102,0.874832,...,0.887095,0.892897,0.894645,0.895387,0.867520,0.965298,0.982779,0.990644,0.992916,0.993862


## Creating X, Y matrices for mapping

In [231]:
#Adding individual latent vectors
for user_id in train:
    source_ind = np.where(new_user_ids_video_6k == user_id)
    target_ind = np.where(new_user_ids_mt_6k == user_id)
    X_train = np.concatenate((X_train, U_video[source_ind]), axis = 0)
    Y_train = np.concatenate((Y_train, U_mt[target_ind]), axis = 0)
    
for user_id in test:
    source_ind = np.where(new_user_ids_video_6k == user_id)
    target_ind = np.where(new_user_ids_mt_6k == user_id)
    X_test = np.concatenate((X_test, U_video[source_ind]), axis = 0)  

    
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(4800, 100)
(4800, 100)
(1200, 100)
(1200, 14972)


In [232]:
groupsToMembers_video = []
groupsToMembers_mt = []
for i in range(K1):
    groupsToMembers_video.append([])
    groupsToMembers_mt.append([])

for i in range(UGidx_video.shape[0]):
    groupsToMembers_video[int(UGidx_video[i][0])].append(new_user_ids_video_6k[i])
    
for i in range(UGidx_mt.shape[0]):
    groupsToMembers_mt[int(UGidx_mt[i][0])].append(new_user_ids_mt_6k[i])
    
print(len(groupsToMembers_video))
print(len(groupsToMembers_mt))

10
10


In [233]:
for i in range(len(groupsToMembers_video)):
    print(len(groupsToMembers_video[i]))

1396
1085
1540
795
375
1219
396
986
829
379


In [234]:
for i in range(len(groupsToMembers_mt)):
    print(len(groupsToMembers_mt[i]))

4125
191
164
194
184
197
176
181
218
170


In [235]:
print(groupsToMembers_mt[5])

['A1T51PSFRXHBIQ', 'A16RCWT59CN1WF', 'A3BAQJHJH2IYL6', 'AMC7VSQ4Z2FPB', 'A13QJY0WVF05ZS', 'A30BM2M59G09SG', 'A21WFNBV74FRV', 'A2Z39DLE2UC7QB', 'A28HJISYLNJH4X', 'A1R4LO2RGQS498', 'A3HA98SL870JKG', 'A2YCWY5GZD4I2U', 'A32C42DD7FAJ08', 'A2W9C1WHTPQ3YG', 'AIBAAIPBT329Y', 'A1LC9TKGZ046K0', 'A3ASKO4R2SZE3D', 'A2VR5DVJR6VFF4', 'A1BLEP8EBACXKC', 'A3EJKPB8J0PL5I', 'A19XQ5VTWGB14Q', 'A2PI3WPMMX2PQ2', 'AVFWQ5Y38LSQL', 'AZQ2GFWNEXGR1', 'A26N0JNNQ0MHU7', 'AHP5XOKMVWD70', 'A31ZB32PN85058', 'A1G5TE2DL25X7X', 'A27Q1X75QCWTID', 'A1PV3NTMSMCB3F', 'AMJFPSZINXHGV', 'A12LU73KOWKCYS', 'AVJOCPMS8KA9A', 'APD7D2W1UMXAE', 'A1HRBJKMQJ89NP', 'A5NUTJCQ322HR', 'A1R4US1NM2XHEB', 'AQK82VFXPWX3O', 'A2FYUQXACR62R0', 'ABIC9QZRBKWKD', 'A3RIL4OEAIIC0S', 'A378IR8MNK2RX2', 'AOI440XY366NN', 'A34NYUMEEFWT0H', 'A1Q8VDQJF06XTU', 'A6WPI8C8GWIRH', 'AXD7G7QG8YN8N', 'A2TW2AS6V4U0DX', 'AHPZDXMEJHC4Q', 'A2P96P7XH7DH19', 'A3BQESQLN9HYSH', 'AI6U12YQES9Z5', 'A3UVKJ3Q3VN4XQ', 'A3OGKONKX1YWIS', 'A2C2NF7MANOP65', 'A229FI0NBBZPL6', 'A193222

In [236]:
counter = 0
for i in range(len(groupsToMembers_video)):
    currGroup_video = groupsToMembers_video[i]
    for j in range(len(groupsToMembers_mt)):
        currGroup_mt = groupsToMembers_mt[j]
        if len(set(currGroup_video).intersection(set(currGroup_mt))) > 0:
            counter = counter + 1
            X_train = np.row_stack((X_train, UG_video[i]))
            Y_train = np.row_stack((Y_train, UG_mt[j]))
            
print(X_train.shape)
print(Y_train.shape)
print(counter)

(4900, 100)
(4900, 100)
100


## MLP Mapping

In [237]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import torch.optim as optim

In [238]:
class GetDataset(torch.utils.data.Dataset):

  def __init__(self, X, y, scale_data=True):
    if not torch.is_tensor(X) and not torch.is_tensor(y):
      self.X = torch.from_numpy(X)
      self.y = torch.from_numpy(y)

  def __len__(self):
      return len(self.X)

  def __getitem__(self, i):
      return self.X[i], self.y[i]

In [239]:
class MLP(nn.Module):
  '''
    Multilayer Perceptron for regression.
  '''
  def __init__(self):
    super().__init__()
    self.layers = nn.Sequential(
      nn.Linear(100, 64),
      nn.ReLU(),
      nn.Linear(64, 32),
      nn.ReLU(),
      nn.Linear(32, 100)
    )


  def forward(self, x):
    '''
      Forward pass
    '''
    return self.layers(x)

In [240]:
# Set fixed random number seed
torch.manual_seed(43)

In [241]:
dataset = GetDataset(X_train, Y_train)
trainloader = torch.utils.data.DataLoader(dataset, batch_size=X_train.shape[0], shuffle=False, num_workers=0)

In [76]:
space = [
    Real(1e-3, 5e-2, name='lr'),
    Real(1e-5, 1e-2, name='decay')
]

In [77]:
def trainMLPAndGetMAE(hyperparameters):
    # Extract hyperparameters
    learning_rate = hyperparameters[0]
    wt_decay = hyperparameters[1]
    
    mlp = MLP()
  
    # Define the loss function and optimizer
    loss_function = nn.MSELoss()
    optimizer = torch.optim.Adam(mlp.parameters(), lr=learning_rate, weight_decay=wt_decay)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=1, verbose=True)
    
    counter = 0
    # Run the training loop
    while optimizer.param_groups[-1]['lr'] > 1e-4: # 5 epochs at maximum

        # Print epoch
    #     print(f'Starting epoch {epoch+1}')

        # Set current loss value
        current_loss = 0.0
        losses = []
        # Iterate over the DataLoader for training data
        for i, data in enumerate(trainloader, 0):

          # Get and prepare inputs
    #         if counter > 2:
    #             break
    #         print(counter)
            inputs, targets = data
            inputs, targets = inputs.float(), targets.float()
            targets = targets.reshape((targets.shape[0], 100))

    #         print("Inputs shape: ", inputs.shape)
    #         print("Target shape: ", targets.shape)
    #         counter += 1

          # Zero the gradients
            optimizer.zero_grad()

          # Perform forward pass
            outputs = mlp(inputs)
    #         print("Output shape: ", outputs.shape)

#             outputs_np = outputs.detach().numpy()
#             targets_np = targets.detach().numpy()
            t_V_mt = torch.from_numpy(V_mt.T).float()

#             t_vec = targets_np@V_mt.T
#             o_vec = outputs_np@V_mt.T
            t_vec = torch.matmul(targets, t_V_mt)
            o_vec = torch.matmul(outputs, t_V_mt)
    #         print("t_vec.shape: ", t_vec.shape)
    #         print("o_vec.shape: ", o_vec.shape)
            indicator_matrix = (t_vec >= 0.7)
    #         print("indicator matrix: ", indicator_matrix)
    #         print(np.all(indicator_matrix == True))
#             t_vec_interacted = torch.from_numpy(t_vec*indicator_matrix).requires_grad_() 
#             o_vec_interacted = torch.from_numpy(o_vec*indicator_matrix).requires_grad_() 
            t_vec_interacted = torch.mul(t_vec, indicator_matrix)
            o_vec_interacted = torch.mul(o_vec, indicator_matrix)
#             t = torch.from_numpy(a)

          # Compute loss
            loss = loss_function(o_vec_interacted, t_vec_interacted)
    #         print("Loss: ", loss.item())
            print("Epoch : " + str(counter + 1) + " | Error : " + str(loss.item()) + " | LR : " + str(optimizer.param_groups[-1]['lr']))
            losses.append(loss.item())
          # Perform backward pass
            loss.backward()

          # Perform optimization
            optimizer.step()


        mean_loss = sum(losses) / len(losses)
        scheduler.step(mean_loss)
#         print(f"Loss at epoch {counter + 1} = {mean_loss}")
        if optimizer.param_groups[-1]['lr'] <= 1e-4:
            break;
        counter += 1

    # Process is complete.
    print('Training process has finished.')
    
    pred_X_train = mlp(torch.from_numpy(np.float32(X_train))).detach().numpy()
    print("pred_X_train: ", pred_X_train.shape)
    par1 = pred_X_train@V_mt.T
    par2 = Y_train@V_mt.T
    mae = MAE(par1, par2)
    print("MAE: ", mae)
    print()
    print()
    return mae

In [78]:
def objective(hyperparameters):
    print(hyperparameters)
    return trainMLPAndGetMAE(hyperparameters)

In [79]:
# Perform Bayesian optimization
result = gp_minimize(objective, space, n_calls=20)

[0.04638241794040354, 0.0037714444909296876]
Epoch : 1 | Error : 9.492766380310059 | LR : 0.04638241794040354
Epoch : 2 | Error : 14.33516788482666 | LR : 0.04638241794040354
Epoch : 3 | Error : 5.154274940490723 | LR : 0.04638241794040354
Epoch : 4 | Error : 5.667237758636475 | LR : 0.04638241794040354
Epoch : 5 | Error : 4.504531383514404 | LR : 0.04638241794040354
Epoch : 6 | Error : 1.3218625783920288 | LR : 0.04638241794040354
Epoch : 7 | Error : 3.7029924392700195 | LR : 0.04638241794040354
Epoch : 8 | Error : 0.27298635244369507 | LR : 0.04638241794040354
Epoch : 9 | Error : 0.3766518533229828 | LR : 0.04638241794040354
Epoch : 10 | Error : 0.669479489326477 | LR : 0.04638241794040354
Epoch 00010: reducing learning rate of group 0 to 4.6382e-03.
Epoch : 11 | Error : 0.5784799456596375 | LR : 0.004638241794040355
Epoch : 12 | Error : 0.5397768020629883 | LR : 0.004638241794040355
Epoch 00012: reducing learning rate of group 0 to 4.6382e-04.
Epoch : 13 | Error : 0.4789561331272125

Epoch : 3 | Error : 4.179182052612305 | LR : 0.037432120529618224
Epoch : 4 | Error : 1.5124704837799072 | LR : 0.037432120529618224
Epoch 00004: reducing learning rate of group 0 to 3.7432e-03.
Epoch : 5 | Error : 0.916900634765625 | LR : 0.0037432120529618225
Epoch : 6 | Error : 0.7709575295448303 | LR : 0.0037432120529618225
Epoch : 7 | Error : 0.4832952916622162 | LR : 0.0037432120529618225
Epoch : 8 | Error : 0.24945053458213806 | LR : 0.0037432120529618225
Epoch : 9 | Error : 0.13151374459266663 | LR : 0.0037432120529618225
Epoch : 10 | Error : 0.11396452784538269 | LR : 0.0037432120529618225
Epoch : 11 | Error : 0.15284158289432526 | LR : 0.0037432120529618225
Epoch : 12 | Error : 0.20825836062431335 | LR : 0.0037432120529618225
Epoch 00012: reducing learning rate of group 0 to 3.7432e-04.
Epoch : 13 | Error : 0.2549085319042206 | LR : 0.0003743212052961823
Epoch : 14 | Error : 0.25750550627708435 | LR : 0.0003743212052961823
Epoch 00014: reducing learning rate of group 0 to 3.7

Epoch : 1 | Error : 8.495993614196777 | LR : 0.01860883725770185
Epoch : 2 | Error : 0.9308218359947205 | LR : 0.01860883725770185
Epoch : 3 | Error : 23.076810836791992 | LR : 0.01860883725770185
Epoch : 4 | Error : 0.9904090762138367 | LR : 0.01860883725770185
Epoch 00004: reducing learning rate of group 0 to 1.8609e-03.
Epoch : 5 | Error : 1.6987203359603882 | LR : 0.001860883725770185
Epoch : 6 | Error : 1.942606806755066 | LR : 0.001860883725770185
Epoch 00006: reducing learning rate of group 0 to 1.8609e-04.
Epoch : 7 | Error : 2.113466501235962 | LR : 0.00018608837257701852
Epoch : 8 | Error : 2.12420916557312 | LR : 0.00018608837257701852
Epoch 00008: reducing learning rate of group 0 to 1.8609e-05.
Training process has finished.
pred_X_train:  (4900, 100)
MAE:  1.413898210608661


[0.024062812984902072, 0.009274990361304065]
Epoch : 1 | Error : 10.026557922363281 | LR : 0.024062812984902072
Epoch : 2 | Error : 0.7692903876304626 | LR : 0.024062812984902072
Epoch : 3 | Error : 

In [80]:
# Get the best hyperparameters and performance
# best_hyperparameters = {
#     'K1' = result.x[0]
#     'K2' = result.x[1]
#     'lambda1UnifiedLS' = result.x[2]
#     'lambda3UnifiedLS' = result.x[3]
# }
best_performance = result.fun
print("Best MAE: {:.4f}".format(result.fun))
print("Best Hyperparameters: {}".format(dict(zip(['lr', 'decay'], result.x))))

Best MAE: 0.2653
Best Hyperparameters: {'lr': 0.03536633914220312, 'decay': 0.007966155876772798}


In [81]:
# Print the MAE values and hyperparameters for each call
for i, val in enumerate(result.func_vals):
    print("Call {}: MAE={:.4f}, Hyperparameters={}".format(i, val, result.x_iters[i]))

Call 0: MAE=0.6247, Hyperparameters=[0.04638241794040354, 0.0037714444909296876]
Call 1: MAE=0.7772, Hyperparameters=[0.014845258857330436, 0.0033409321435141238]
Call 2: MAE=2.2831, Hyperparameters=[0.03888603692713841, 0.002264185482115058]
Call 3: MAE=1.2267, Hyperparameters=[0.003409584969585868, 0.004142875449376066]
Call 4: MAE=0.9921, Hyperparameters=[0.005066822701456474, 0.00738277850529626]
Call 5: MAE=0.3611, Hyperparameters=[0.02276202325139714, 0.009280140084071865]
Call 6: MAE=1.0436, Hyperparameters=[0.0021413466464740754, 0.008122409778999336]
Call 7: MAE=0.4312, Hyperparameters=[0.037432120529618224, 0.008028566815463474]
Call 8: MAE=0.2653, Hyperparameters=[0.03536633914220312, 0.007966155876772798]
Call 9: MAE=0.6806, Hyperparameters=[0.006640519505238876, 0.0022595552370384903]
Call 10: MAE=0.8828, Hyperparameters=[0.02906641678884884, 0.008206966550712357]
Call 11: MAE=1.6771, Hyperparameters=[0.03652554615430099, 0.007808278103590817]
Call 12: MAE=0.5632, Hyperpar

In [82]:
# Print the values of the optimized hyperparameters
print('Optimized Hyperparameters:')
for hyperparameter, value in zip(space, result.x):
    print('{}: {}'.format(hyperparameter.name, value))

Optimized Hyperparameters:
lr: 0.03536633914220312
decay: 0.007966155876772798


## MLP with tuned hyperparameters

In [261]:
torch.manual_seed(42)

In [262]:
mlp = MLP()
  
# Define the loss function and optimizer
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.03536633914220312, weight_decay=0.007966155876772798)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=1, verbose=True)

In [263]:
counter = 0
# Run the training loop
while optimizer.param_groups[-1]['lr'] > 1e-4: # 5 epochs at maximum
    
    # Print epoch
#     print(f'Starting epoch {epoch+1}')
    
    # Set current loss value
    current_loss = 0.0
    losses = []
    # Iterate over the DataLoader for training data
    for i, data in enumerate(trainloader, 0):
      
      # Get and prepare inputs
#         if counter > 2:
#             break
#         print(counter)
        inputs, targets = data
        inputs, targets = inputs.float(), targets.float()
        targets = targets.reshape((targets.shape[0], 100))
        
#         print("Inputs shape: ", inputs.shape)
#         print("Target shape: ", targets.shape)
#         counter += 1
      
      # Zero the gradients
        optimizer.zero_grad()
      
      # Perform forward pass
        outputs = mlp(inputs)
#         print("Output shape: ", outputs.shape)
        
#         outputs_np = outputs.detach().numpy()
#         targets_np = targets.detach().numpy()
        t_V_mt = torch.from_numpy(V_mt.T).float()
    
#         t_vec = targets_np@V_mt.T
#         o_vec = outputs_np@V_mt.T
#         print("t_vec.shape: ", t_vec.shape)
#         print("o_vec.shape: ", o_vec.shape)
        t_vec = torch.matmul(targets, t_V_mt)
        o_vec = torch.matmul(outputs, t_V_mt)
        indicator_matrix = (t_vec >= 0.7)
#         print("indicator matrix: ", indicator_matrix)
#         print(np.all(indicator_matrix == True))
#         t_vec_interacted = t_vec*indicator_matrix
#         o_vec_interacted = o_vec*indicator_matrix
        t_vec_interacted = torch.mul(t_vec, indicator_matrix)
        o_vec_interacted = torch.mul(o_vec, indicator_matrix)
#             t = torch.from_numpy(a)

        # Compute loss
        loss = loss_function(o_vec_interacted, t_vec_interacted)
        
#         print("Loss: ", loss.item())
        print("Epoch : " + str(counter + 1) + " | Error : " + str(loss.item()) + " | LR : " + str(optimizer.param_groups[-1]['lr']))
        losses.append(loss.item())
      # Perform backward pass
        loss.backward()
      
      # Perform optimization
        optimizer.step()
        
        
    mean_loss = sum(losses) / len(losses)
    scheduler.step(mean_loss)
    print(f"Loss at epoch {counter + 1} = {mean_loss}")
    if optimizer.param_groups[-1]['lr'] <= 1e-4:
        break;
    counter += 1

# Process is complete.
print('Training process has finished.')

Epoch : 1 | Error : 12.009551048278809 | LR : 0.03536633914220312
Loss at epoch 1 = 12.009551048278809
Epoch : 2 | Error : 2.498908281326294 | LR : 0.03536633914220312
Loss at epoch 2 = 2.498908281326294
Epoch : 3 | Error : 5.8115763664245605 | LR : 0.03536633914220312
Loss at epoch 3 = 5.8115763664245605
Epoch : 4 | Error : 4.745058536529541 | LR : 0.03536633914220312
Epoch 00004: reducing learning rate of group 0 to 3.5366e-03.
Loss at epoch 4 = 4.745058536529541
Epoch : 5 | Error : 2.18630313873291 | LR : 0.0035366339142203123
Loss at epoch 5 = 2.18630313873291
Epoch : 6 | Error : 1.8908532857894897 | LR : 0.0035366339142203123
Loss at epoch 6 = 1.8908532857894897
Epoch : 7 | Error : 1.5729074478149414 | LR : 0.0035366339142203123
Loss at epoch 7 = 1.5729074478149414
Epoch : 8 | Error : 1.2815881967544556 | LR : 0.0035366339142203123
Loss at epoch 8 = 1.2815881967544556
Epoch : 9 | Error : 1.0702341794967651 | LR : 0.0035366339142203123
Loss at epoch 9 = 1.0702341794967651
Epoch : 1

In [264]:
pred_X_train = mlp(torch.from_numpy(np.float32(X_train))).detach().numpy()
print(pred_X_train.shape)
cutoff = 3
k = 40
par1 = pred_X_train@V_mt.T
par2 = Y_train@V_mt.T
# mae,rmse = EvaluationAllUpdated_N(par1, par2, k, cutoff)
# print("Training Metrics")
# print(mae, rmse)

(4900, 100)


In [265]:
mae,rmse = EvaluationAllUpdated_N(par1, par2, k, cutoff)
Training_Metrics                = ["CDGRS_Train",mae,rmse]
Result_Trn_CDR.loc[len(Result_Trn_CDR)]       = ["Train-3",mae,rmse]

In [266]:
pred_X_test = mlp(torch.from_numpy(np.float32(X_test))).detach().numpy()
print(pred_X_test.shape)
cutoff = 3
k = 40
par1 = pred_X_test@V_mt.T
par1[par1 > 5] = 5
par2 = Y_test
# mae,rmse = EvaluationAllUpdated_N(par1, par2, k, cutoff)
# print("Test Metrics")
# print(mae, rmse)

(1200, 100)


In [267]:
mae,rmse = EvaluationAllUpdated_N(par1, par2, k, cutoff)
Testing_Metrics                = ["CDGRS_Test",mae,rmse]
Result_Tst_CDR.loc[len(Result_Tst_CDR)]       = ["Test-3",mae,rmse]

In [268]:
Result_Trn_CDR

,Model,MAE,RMSE
0,Train-1,0.260722,0.326714
1,Train-2,0.636292,0.724106
2,Train-3,0.817112,1.146939


In [269]:
Result_Trn_CDR.mean()

C:\Users\NIT\AppData\Local\Temp\ipykernel_30256\3129751611.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  Result_Trn_CDR.mean()


MAE     0.571375
RMSE    0.732586
dtype: float64

In [270]:
Result_Tst_CDR

,Model,MAE,RMSE
0,Test-1,1.181412,1.357778
1,Test-2,1.646056,1.828531
2,Test-3,1.077139,1.327495


In [271]:
Result_Tst_CDR.mean()

C:\Users\NIT\AppData\Local\Temp\ipykernel_30256\661790523.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  Result_Tst_CDR.mean()


MAE     1.301535
RMSE    1.504601
dtype: float64

In [196]:
# Result_Trn_CDR.drop(index=Result_Trn_CDR.index[-1],axis=0,inplace=True)
# Result_Tst_CDR.drop(index=Result_Tst_CDR.index[-1],axis=0,inplace=True)